### Pipeline to Fine-Tune CAMELBERT and Evaluate It Using Three Datasets

This document outlines the pipeline to fine-tune the MARBERT model and evaluate its performance on three distinct datasets. After fine-tuning, we will compare the results across the datasets to assess performance consistency and accuracy across varying input types and data structures.

---

#### Overview of Datasets:
1. **Dataset 1:** GPT-4o-generated samples
2. **Dataset 2:** Samples classified using 18 binary classifiers on random samples
3. **Dataset 3:** Samples classified using 18 binary classifiers on equivalent samples

---

In [1]:
import torch
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    EvalPrediction,
    TrainerCallback,
)
from sklearn.metrics import accuracy_score, hamming_loss, precision_recall_fscore_support
from preprocess import final_eliminations
from transformers import TrainingArguments, EarlyStoppingCallback
from transformers.trainer_utils import IntervalStrategy


/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
class BertTrainer:
    def __init__(self, training_dataset_path, labels, exp_num, stage = 0, threshold=0.5, model_name="CAMeL-Lab/bert-base-arabic-camelbert-ca"):
        self.labels = labels
        self.label2id = {label: idx for idx, label in enumerate(labels)}
        self.id2label = {idx: label for label, idx in self.label2id.items()}
        self.model_name = model_name
        self.exp_num = exp_num
        training_dataset = pd.read_csv(training_dataset_path)
        self.training_dataset_processed = pd.DataFrame({
            'text': training_dataset['tweet'],
            'label': training_dataset[self.labels].values.tolist()
        })
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
        self.train_df, self.val_df = train_test_split(self.training_dataset_processed, test_size=0.1, random_state=42)
        self.train_df['text'] = self.train_df['text'].astype(str)
        self.val_df['text'] = self.val_df['text'].astype(str)
        self.train_dataset = self.create_dataset(self.train_df)
        self.val_dataset = self.create_dataset(self.val_df)
        self.threshold = threshold
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model = self.load_model(dropout_rate=0.3)  # Adding dropout rate
        self.stage = stage

    def create_dataset(self, df):
        encodings = self.tokenizer(
            df['text'].tolist(), truncation=True, padding=True, max_length=128
        )
        return TweetDataset(encodings, df['label'].values)

    def load_model(self, dropout_rate=0.3):
        model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=len(self.labels),
            id2label=self.id2label,
            label2id=self.label2id,
            problem_type="multi_label_classification"
        )
       
        model.config.hidden_dropout_prob = dropout_rate
        model.config.attention_probs_dropout_prob = dropout_rate
        
        for param in model.bert.encoder.layer[:8].parameters():
            param.requires_grad = False

        model.to(self.device)
        return model

    
    def predict(self, texts):
        encodings = self.tokenizer(
            texts, 
            truncation=True, 
            padding=True, 
            max_length=128, 
            return_tensors="pt"
        )
        input_ids = encodings['input_ids'].to(self.device)
        attention_mask = encodings['attention_mask'].to(self.device)
        with torch.no_grad():
            outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
        probabilities = torch.sigmoid(logits).cpu().numpy()
        predictions = (probabilities >= self.threshold).astype(int)
        variation_score = 1 - (np.sum(probabilities)/18)
        return predictions, probabilities, variation_score

    
    def evaluate(self, dev_path):
        if '.tsv' in dev_path:
            dev = pd.read_csv(dev_path, sep='\t')
        else:
            dev = pd.read_csv(dev_path)
        
        dev = final_eliminations(dev, column_name="sentence")

        df_replaced = dev.replace({'y': 1, 'n': 0})
        country_columns = df_replaced.columns.difference(['sentence'])
        df_replaced['label'] = df_replaced[country_columns].values.tolist()
        df_final = df_replaced[['sentence', 'label']]
        
        predictions, probabilities, _ = self.predict(df_final['sentence'].tolist())
        output_dir = f'./exp_{self.exp_num}'
        output_file = os.path.join(self.save_dir, f"{self.model_name.replace('/', '-')}-experiment-{self.exp_num}_predictions.txt")
        os.makedirs(output_dir, exist_ok=True)
        with open(output_file, 'w') as f:
            for i, (pred, prob) in enumerate(zip(predictions, probabilities)):
                pred_str = ','.join(map(str, pred))
                f.write(f'{pred_str}\n')

        
        indexes = [0, 2, 4, 10, 13, 14, 15, 17]
        predictions = [output[indexes] for output in predictions]


        subset_accuracy = accuracy_score(df_final['label'].tolist(), predictions)
        print(f"Subset Accuracy: {subset_accuracy:.4f}")

        hamming = hamming_loss(df_final['label'].tolist(), predictions)
        print(f"Hamming Loss: {hamming:.4f}")

        precision, recall, f1, _ = precision_recall_fscore_support(
            df_final['label'].tolist(), predictions, average='micro'  
        )
        print(f"Micro Precision: {precision:.4f}")
        print(f"Micro Recall: {recall:.4f}")
        print(f"Micro F1-Score: {f1:.4f}")

        precision_per_label, recall_per_label, f1_per_label, _ = precision_recall_fscore_support(
            df_final['label'].tolist(), predictions, average=None 
        )
        print(f"Precision per label: {precision_per_label}")
        print(f"Recall per label: {recall_per_label}")
        print(f"F1-Score per label: {f1_per_label}")
        multilabel_check = [np.sum(np.array(prediction)) for prediction in predictions]
        print(set(multilabel_check))


    def compute_metrics(self, p: EvalPrediction):
        preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
        result = self.multi_label_metrics(preds, p.label_ids)
        return result

    def multi_label_metrics(self, predictions, labels):
        sigmoid = torch.nn.Sigmoid()
        probs = sigmoid(torch.Tensor(predictions))
        y_pred = np.zeros(probs.shape)
        y_pred[np.where(probs >= self.threshold)] = 1
        f1 = f1_score(labels, y_pred, average='micro')
        roc_auc = roc_auc_score(labels, y_pred, average='micro')
        accuracy = accuracy_score(labels, y_pred)
        return {'f1': f1, 'roc_auc': roc_auc, 'accuracy': accuracy}
    
    def train(
        self,
        num_train_epochs=3,  
        metric_for_best_model="eval_f1",  
        greater_is_better=True,  
        per_device_train_batch_size=8,
        per_device_eval_batch_size=16,
        patience=2,
        warmup_steps=500,  
        base_learning_rate=5e-5,  
    ):
        training_args = TrainingArguments(
            output_dir='./exp_' + str(self.exp_num) + '/results',
            num_train_epochs=num_train_epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_eval_batch_size,
            warmup_steps=warmup_steps,
            learning_rate=base_learning_rate,
            weight_decay=0.01,
            logging_dir='./exp_' + str(self.exp_num) + '/logs',
            logging_steps=500,
            evaluation_strategy="epoch",
            save_strategy="epoch",
            load_best_model_at_end=True,
            metric_for_best_model=metric_for_best_model,
            greater_is_better=greater_is_better,
            fp16=True,
            report_to=["tensorboard"],
            lr_scheduler_type="linear",  
        )

        early_stopping_callback = EarlyStoppingCallback(
            early_stopping_patience=patience
        )

        trainer = CustomTrainer(
            model=self.model,
            args=training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.val_dataset,
            tokenizer=self.tokenizer,
            compute_metrics=self.compute_metrics,
            callbacks=[early_stopping_callback]  
        )

        trainer.train()

        
        best_metric_value = trainer.state.best_metric 
        num_epochs = training_args.num_train_epochs
        greater_is_better = training_args.greater_is_better
        metric_name = training_args.metric_for_best_model
        save_dir = f'./exp_{self.exp_num}/stage_{self.stage}'
        self.save_dir = save_dir
        os.makedirs(save_dir, exist_ok=True)
        self.model.save_pretrained(save_dir, safe_serialization=False)
        self.tokenizer.save_pretrained(save_dir)

        


class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.float)
        return item


class CustomTrainer(Trainer):
    def save_model(self, output_dir=None, **kwargs):
        if output_dir is None:
            output_dir = self.args.output_dir
        for param in self.model.parameters():
            param.data = param.data.contiguous()
        super().save_model(output_dir, **kwargs)


In [3]:
dataset_path = 'Project/NADI2024/subtask1/multilabel/NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv'
df = pd.read_csv(dataset_path)


In [5]:
import pandas as pd
from sklearn.utils import shuffle

label_columns = df.columns[2:-1]  
df[label_columns] = df[label_columns].astype(int)  

threshold = 500  
balanced_df = pd.DataFrame()



for num_classes in range(1, len(label_columns) + 1):
    subset = df[df[label_columns].sum(axis=1) == num_classes]  
    
   
    if len(subset) > threshold:
        subset = shuffle(subset).head(threshold)
    
    
    balanced_df = pd.concat([balanced_df, subset], ignore_index=True)


balanced_df = shuffle(balanced_df).reset_index(drop=True)
balanced_df.to_csv('balanced_multilabel_dataset_' + str(threshold) + '.csv', index=False)

print("Balanced dataset created and saved as 'balanced_multilabel_dataset_500.csv'")

label_columns = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait', 'Lebanon', 
                 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar', 'Saudi_Arabia', 
                 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']


counts = {}


for i in range(19):
    counts[i] = (balanced_df[label_columns].sum(axis=1) == i).sum()


Balanced dataset created and saved as 'balanced_multilabel_dataset_500.csv'


In [6]:
counts

{0: 0,
 1: 500,
 2: 500,
 3: 500,
 4: 500,
 5: 500,
 6: 500,
 7: 500,
 8: 500,
 9: 500,
 10: 168,
 11: 137,
 12: 334,
 13: 273,
 14: 256,
 15: 500,
 16: 11,
 17: 26,
 18: 500}

### EXPERIMENT 6

In [161]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv"]
dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[3]}"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    labels=labels,
    threshold=0.3,
    exp_num=6
)
trainer.train(
    num_train_epochs=10,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=24
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-ca and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  5%|▍         | 502/10060 [00:27<08:58, 17.75it/s]

{'loss': 0.5952, 'grad_norm': 1.13016939163208, 'learning_rate': 4.97e-05, 'epoch': 0.5}


 10%|▉         | 1004/10060 [00:56<07:53, 19.14it/s]

{'loss': 0.5852, 'grad_norm': 4.487825870513916, 'learning_rate': 4.7400627615062765e-05, 'epoch': 0.99}


 10%|█         | 1006/10060 [00:56<08:19, 18.13it/s]

...

{'loss': 0.1922, 'grad_norm': 2.906043529510498, 'learning_rate': 3.3995815899581595e-07, 'epoch': 9.94}


100%|██████████| 10060/10060 [10:35<00:00, 16.57it/s]

{'eval_loss': 0.7048290967941284, 'eval_f1': 0.7522365805168986, 'eval_roc_auc': 0.7531335047226225, 'eval_accuracy': 0.09821428571428571, 'eval_runtime': 0.3158, 'eval_samples_per_second': 1418.587, 'eval_steps_per_second': 60.163, 'epoch': 10.0}


100%|██████████| 10060/10060 [10:38<00:00, 15.75it/s]


{'train_runtime': 638.6339, 'train_samples_per_second': 62.994, 'train_steps_per_second': 15.752, 'train_loss': 0.36358296686327957, 'epoch': 10.0}


In [165]:
trainer.evaluate(dev_path=dev_path)

Subset Accuracy: 0.0833
Hamming Loss: 0.3354
Micro Precision: 0.5590
Micro Recall: 0.4522
Micro F1-Score: 0.5000
Precision per label: [0.42857143 0.45       0.53731343 0.62711864 0.71428571 0.50847458
 0.25       0.63934426]
Recall per label: [0.08571429 0.23076923 0.72       0.578125   0.11904762 0.65217391
 0.0952381  0.66101695]
F1-Score per label: [0.14285714 0.30508475 0.61538462 0.60162602 0.20408163 0.57142857
 0.13793103 0.65      ]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(8)}


/tmp/ipykernel_4025847/3782111259.py:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [166]:
import os

scorer_script = "/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_6/camelbert_finetuned_epochs_10_eval_f1_0.7645_greater_threshold_0.3/CAMeL-Lab-bert-base-arabic-camelbert-ca-experiment-6_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 51.94 %
MACRO AVERAGE RECALL SCORE: 39.28 %
MACRO AVERAGE F1-SCORE: 40.35 %
MACRO AVERAGE ACCURACY: 66.46 %



### EXPERIMENT 7

In [171]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = f"/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dev_path = "/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-mix",
    labels=labels,
    threshold=0.3,
    exp_num=7
)
trainer.train(
    num_train_epochs=1,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 252/252 [00:21<00:00, 13.91it/s]

{'eval_loss': 0.48796308040618896, 'eval_f1': 0.7621827981136312, 'eval_roc_auc': 0.7501663991770002, 'eval_accuracy': 0.12667660208643816, 'eval_runtime': 0.4998, 'eval_samples_per_second': 1342.658, 'eval_steps_per_second': 56.027, 'epoch': 1.0}


100%|██████████| 252/252 [00:24<00:00, 10.39it/s]


{'train_runtime': 24.2469, 'train_samples_per_second': 248.857, 'train_steps_per_second': 10.393, 'train_loss': 0.5769250052315849, 'epoch': 1.0}
Subset Accuracy: 0.1250
Hamming Loss: 0.2844
Micro Precision: 0.6005
Micro Recall: 0.6966
Micro F1-Score: 0.6450
Precision per label: [0.78571429 0.675      0.49484536 0.65384615 0.88235294 0.4875
 0.66666667 0.65277778]
Recall per label: [0.31428571 0.69230769 0.96       0.796875   0.35714286 0.84782609
 0.47619048 0.79661017]
F1-Score per label: [0.44897959 0.6835443  0.65306122 0.71830986 0.50847458 0.61904762
 0.55555556 0.71755725]
{np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(8)}


/tmp/ipykernel_4025847/3782111259.py:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [172]:
import os

scorer_script = "=/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "=/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "=/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_7/camelbert_finetuned_epochs_1_eval_f1_0.7622_greater_threshold_0.3/CAMeL-Lab-bert-base-arabic-camelbert-mix-experiment-7_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 66.23 %
MACRO AVERAGE RECALL SCORE: 65.52 %
MACRO AVERAGE F1-SCORE: 61.31 %
MACRO AVERAGE ACCURACY: 71.56 %



### EXPERIMENT 8

In [173]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-mix",
    labels=labels,
    threshold=0.3,
    exp_num=8
)
trainer.train(
    num_train_epochs=2,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                                 
 50%|█████     | 252/504 [00:19<00:19, 13.26it/s]

{'eval_loss': 0.48796144127845764, 'eval_f1': 0.7621827981136312, 'eval_roc_auc': 0.7501663991770002, 'eval_accuracy': 0.12667660208643816, 'eval_runtime': 0.4798, 'eval_samples_per_second': 1398.644, 'eval_steps_per_second': 58.364, 'epoch': 1.0}


100%|█████████▉| 502/504 [00:42<00:00, 12.29it/s]

{'loss': 0.5124, 'grad_norm': 3.563359260559082, 'learning_rate': 4.99e-05, 'epoch': 1.98}


100%|██████████| 504/504 [00:45<00:00, 12.74it/s]

{'eval_loss': 0.4665817618370056, 'eval_f1': 0.769424942809813, 'eval_roc_auc': 0.7668044404020513, 'eval_accuracy': 0.14754098360655737, 'eval_runtime': 0.4957, 'eval_samples_per_second': 1353.615, 'eval_steps_per_second': 56.485, 'epoch': 2.0}


100%|██████████| 504/504 [00:47<00:00, 10.53it/s]


{'train_runtime': 47.8604, 'train_samples_per_second': 252.15, 'train_steps_per_second': 10.531, 'train_loss': 0.5122848169671165, 'epoch': 2.0}
Subset Accuracy: 0.1333
Hamming Loss: 0.2646
Micro Precision: 0.6321
Micro Recall: 0.6854
Micro F1-Score: 0.6577
Precision per label: [0.6875     0.70833333 0.52808989 0.65384615 1.         0.53846154
 0.64705882 0.75510204]
Recall per label: [0.31428571 0.87179487 0.94       0.796875   0.26190476 0.91304348
 0.52380952 0.62711864]
F1-Score per label: [0.43137255 0.7816092  0.67625899 0.71830986 0.41509434 0.67741935
 0.57894737 0.68518519]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)}


/tmp/ipykernel_4025847/3782111259.py:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [174]:
import os

scorer_script = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_8/camelbert_finetuned_epochs_2_eval_f1_0.7694_greater_threshold_0.3/CAMeL-Lab-bert-base-arabic-camelbert-mix-experiment-8_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 68.98 %
MACRO AVERAGE RECALL SCORE: 65.61 %
MACRO AVERAGE F1-SCORE: 62.05 %
MACRO AVERAGE ACCURACY: 73.54 %



### EXPERIMENT 9

In [175]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_8/camelbert_finetuned_epochs_2_eval_f1_0.7694_greater_threshold_0.3",
    labels=labels,
    threshold=0.3,
    exp_num=9
)
trainer.train(
    num_train_epochs=2,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)

/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 50%|█████     | 252/504 [00:19<00:18, 13.85it/s]

{'eval_loss': 0.4918179214000702, 'eval_f1': 0.7811874048553802, 'eval_roc_auc': 0.7815018201807157, 'eval_accuracy': 0.16989567809239942, 'eval_runtime': 0.495, 'eval_samples_per_second': 1355.615, 'eval_steps_per_second': 56.568, 'epoch': 1.0}


100%|█████████▉| 502/504 [00:41<00:00, 13.14it/s]

{'loss': 0.3169, 'grad_norm': 3.382401466369629, 'learning_rate': 4.99e-05, 'epoch': 1.98}


100%|██████████| 504/504 [00:44<00:00, 13.63it/s]

{'eval_loss': 0.5153090953826904, 'eval_f1': 0.7765307806568741, 'eval_roc_auc': 0.7810091627168709, 'eval_accuracy': 0.16095380029806258, 'eval_runtime': 0.4839, 'eval_samples_per_second': 1386.632, 'eval_steps_per_second': 57.862, 'epoch': 2.0}


100%|██████████| 504/504 [00:46<00:00, 10.79it/s]


{'train_runtime': 46.7301, 'train_samples_per_second': 258.249, 'train_steps_per_second': 10.785, 'train_loss': 0.3173308519143907, 'epoch': 2.0}
Subset Accuracy: 0.1833
Hamming Loss: 0.2406
Micro Precision: 0.6791
Micro Recall: 0.6657
Micro F1-Score: 0.6723
Precision per label: [0.78571429 0.76744186 0.57746479 0.73134328 0.84210526 0.56060606
 0.66666667 0.74074074]
Recall per label: [0.31428571 0.84615385 0.82       0.765625   0.38095238 0.80434783
 0.47619048 0.6779661 ]
F1-Score per label: [0.44897959 0.80487805 0.67768595 0.7480916  0.52459016 0.66071429
 0.55555556 0.7079646 ]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(8)}


/tmp/ipykernel_4025847/3782111259.py:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [176]:
import os

scorer_script = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_9/camelbert_finetuned_epochs_2_eval_f1_0.7812_greater_threshold_0.3/-home-lara.hassan-Documents-Cross-Country-Dialectal-Arabic-Identification-exp_8-camelbert_finetuned_epochs_2_eval_f1_0.7694_greater_threshold_0.3-experiment-9_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 70.90 %
MACRO AVERAGE RECALL SCORE: 63.57 %
MACRO AVERAGE F1-SCORE: 64.11 %
MACRO AVERAGE ACCURACY: 75.94 %



### EXPERIMENT 10

In [181]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv", "balanced_multilabel_dataset_750.csv"]
dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[5]}"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-mix",
    labels=labels,
    threshold=0.3,
    exp_num=10
)
trainer.train(
    num_train_epochs=6,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 17%|█▋        | 330/1980 [00:26<03:22,  8.15it/s]

{'eval_loss': 0.4739709496498108, 'eval_f1': 0.7463900752491357, 'eval_roc_auc': 0.7755923502913882, 'eval_accuracy': 0.11161731207289294, 'eval_runtime': 0.6115, 'eval_samples_per_second': 1435.763, 'eval_steps_per_second': 60.505, 'epoch': 1.0}


 25%|██▌       | 502/1980 [00:42<02:00, 12.23it/s]

{'loss': 0.5121, 'grad_norm': 3.3443150520324707, 'learning_rate': 4.99e-05, 'epoch': 1.52}


 33%|███▎      | 660/1980 [00:56<02:11, 10.03it/s]

...

100%|██████████| 1980/1980 [03:01<00:00, 13.02it/s]

{'eval_loss': 0.5531436800956726, 'eval_f1': 0.7577061437539677, 'eval_roc_auc': 0.7889758052175457, 'eval_accuracy': 0.15831435079726652, 'eval_runtime': 0.7475, 'eval_samples_per_second': 1174.636, 'eval_steps_per_second': 49.501, 'epoch': 6.0}


100%|██████████| 1980/1980 [03:04<00:00, 10.73it/s]


{'train_runtime': 184.5783, 'train_samples_per_second': 256.704, 'train_steps_per_second': 10.727, 'train_loss': 0.32922791375054256, 'epoch': 6.0}
Subset Accuracy: 0.1583
Hamming Loss: 0.2594
Micro Precision: 0.6801
Micro Recall: 0.5674
Micro F1-Score: 0.6187
Precision per label: [0.69230769 0.775      0.6        0.71929825 0.69230769 0.62264151
 0.46153846 0.77083333]
Recall per label: [0.25714286 0.79487179 0.72       0.640625   0.21428571 0.7173913
 0.28571429 0.62711864]
F1-Score per label: [0.375      0.78481013 0.65454545 0.67768595 0.32727273 0.66666667
 0.35294118 0.69158879]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)}


/tmp/ipykernel_4025847/3782111259.py:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [182]:
import os

scorer_script = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "exp_10/camelbert_finetuned_epochs_6_eval_f1_0.7602_greater_threshold_0.3/CAMeL-Lab-bert-base-arabic-camelbert-mix-experiment-10_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 66.67 %
MACRO AVERAGE RECALL SCORE: 53.21 %
MACRO AVERAGE F1-SCORE: 56.63 %
MACRO AVERAGE ACCURACY: 74.06 %



In [183]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv", "balanced_multilabel_dataset_750.csv"]
dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_10/camelbert_finetuned_epochs_6_eval_f1_0.7602_greater_threshold_0.3",
    labels=labels,
    threshold=0.3,
    exp_num=10
)
trainer.train(
    num_train_epochs=6,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)


/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 17%|█▋        | 252/1512 [00:19<01:30, 13.87it/s]

{'eval_loss': 0.28491029143333435, 'eval_f1': 0.8595482203884305, 'eval_roc_auc': 0.8654463416464854, 'eval_accuracy': 0.28315946348733234, 'eval_runtime': 0.4594, 'eval_samples_per_second': 1460.564, 'eval_steps_per_second': 60.948, 'epoch': 1.0}


 33%|███▎      | 502/1512 [00:41<01:19, 12.73it/s]

{'loss': 0.2935, 'grad_norm': 3.5294675827026367, 'learning_rate': 5e-05, 'epoch': 1.98}


 33%|███▎      | 504/1512 [00:41<01:15, 13.29it/s]

...

100%|██████████| 1512/1512 [02:13<00:00, 13.59it/s]

{'eval_loss': 0.35399869084358215, 'eval_f1': 0.8584212747994935, 'eval_roc_auc': 0.8651041609176625, 'eval_accuracy': 0.30998509687034276, 'eval_runtime': 0.5414, 'eval_samples_per_second': 1239.478, 'eval_steps_per_second': 51.722, 'epoch': 6.0}


100%|██████████| 1512/1512 [02:16<00:00, 11.11it/s]


{'train_runtime': 136.0435, 'train_samples_per_second': 266.121, 'train_steps_per_second': 11.114, 'train_loss': 0.21364403985164784, 'epoch': 6.0}
Subset Accuracy: 0.1667
Hamming Loss: 0.2687
Micro Precision: 0.6960
Micro Recall: 0.4888
Micro F1-Score: 0.5743
Precision per label: [0.69230769 0.74358974 0.63265306 0.73170732 0.76923077 0.65789474
 0.5        0.76744186]
Recall per label: [0.25714286 0.74358974 0.62       0.46875    0.23809524 0.54347826
 0.33333333 0.55932203]
F1-Score per label: [0.375      0.74358974 0.62626263 0.57142857 0.36363636 0.5952381
 0.4        0.64705882]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6)}


/tmp/ipykernel_4025847/3782111259.py:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [184]:
import os

scorer_script = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_10/camelbert_finetuned_epochs_6_eval_f1_0.8622_greater_threshold_0.3/-home-lara.hassan-Documents-Cross-Country-Dialectal-Arabic-Identification-exp_10-camelbert_finetuned_epochs_6_eval_f1_0.7602_greater_threshold_0.3-experiment-10_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 68.69 %
MACRO AVERAGE RECALL SCORE: 47.05 %
MACRO AVERAGE F1-SCORE: 54.03 %
MACRO AVERAGE ACCURACY: 73.12 %



### EXPERIMENT 11

In [186]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_750.csv"]
dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-mix",
    labels=labels,
    threshold=0.3,
    exp_num=11
)
trainer.train(
    num_train_epochs=4,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 13%|█▎        | 503/3952 [00:27<02:59, 19.22it/s]

{'loss': 0.5519, 'grad_norm': 1.2335233688354492, 'learning_rate': 4.97e-05, 'epoch': 0.51}


                                                  
 25%|██▌       | 988/3952 [00:54<02:33, 19.25it/s]

{'eval_loss': 0.48687317967414856, 'eval_f1': 0.7363865383373167, 'eval_roc_auc': 0.7617819014921697, 'eval_accuracy': 0.12414578587699317, 'eval_runtime': 0.608, 'eval_samples_per_second': 1444.047, 'eval_steps_per_second': 60.854, 'epoch': 1.0}


 25%|██▌       | 1002/3952 [00:57<05:29,  8.95it/s]

...

                                                   
100%|██████████| 3952/3952 [04:05<00:00, 18.63it/s]

{'eval_loss': 0.5275660753250122, 'eval_f1': 0.7625708091474929, 'eval_roc_auc': 0.7929900864074229, 'eval_accuracy': 0.17198177676537585, 'eval_runtime': 0.6341, 'eval_samples_per_second': 1384.718, 'eval_steps_per_second': 58.354, 'epoch': 4.0}


100%|██████████| 3952/3952 [04:08<00:00, 15.93it/s]


{'train_runtime': 248.0563, 'train_samples_per_second': 127.342, 'train_steps_per_second': 15.932, 'train_loss': 0.3670956538273738, 'epoch': 4.0}
Subset Accuracy: 0.1833
Hamming Loss: 0.2573
Micro Precision: 0.6860
Micro Recall: 0.5646
Micro F1-Score: 0.6194
Precision per label: [0.75       0.74418605 0.5862069  0.72222222 0.84615385 0.61538462
 0.46666667 0.80434783]
Recall per label: [0.25714286 0.82051282 0.68       0.609375   0.26190476 0.69565217
 0.33333333 0.62711864]
F1-Score per label: [0.38297872 0.7804878  0.62962963 0.66101695 0.4        0.65306122
 0.38888889 0.7047619 ]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(8)}


/tmp/ipykernel_4025847/3782111259.py:79: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [187]:
import os

scorer_script = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_11/camelbert_finetuned_epochs_4_eval_f1_0.7626_greater_threshold_0.3/CAMeL-Lab-bert-base-arabic-camelbert-mix-experiment-11_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 69.19 %
MACRO AVERAGE RECALL SCORE: 53.56 %
MACRO AVERAGE F1-SCORE: 57.51 %
MACRO AVERAGE ACCURACY: 74.27 %



### EXPERIMENT 12

In [199]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_9/camelbert_finetuned_epochs_2_eval_f1_0.7812_greater_threshold_0.3",
    labels=labels,
    threshold=0.3,
    exp_num=12
)
trainer.train(
    num_train_epochs=5,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)

/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(

  5%|▌         | 52/988 [02:54<00:52, 17.78it/s]  

{'loss': 0.3145, 'grad_norm': 3.7815072536468506, 'learning_rate': 4.99e-05, 'epoch': 0.99}









                                         
                                                  
  5%|▌         | 52/988 [02:55<00:52, 17.78it/s]


{'eval_loss': 0.5411572456359863, 'eval_f1': 0.7787280360752604, 'eval_roc_auc': 0.7744481507891129, 'eval_accuracy': 0.16393442622950818, 'eval_runtime': 0.4921, 'eval_samples_per_second': 1363.594, 'eval_steps_per_second': 56.901, 'epoch': 1.0}



  5%|▌         | 52/988 [03:26<00:52, 17.78it/s]  

...








                                         
                                                   
  5%|▌         | 52/988 [05:07<00:52, 17.78it/s]


{'eval_loss': 0.599375307559967, 'eval_f1': 0.7798488664987405, 'eval_roc_auc': 0.7867945833894436, 'eval_accuracy': 0.15350223546944858, 'eval_runtime': 0.4795, 'eval_samples_per_second': 1399.392, 'eval_steps_per_second': 58.395, 'epoch': 5.0}



100%|██████████| 2515/2515 [02:44<00:00, 15.27it/s]


{'train_runtime': 164.7329, 'train_samples_per_second': 183.145, 'train_steps_per_second': 15.267, 'train_loss': 0.24653972858937076, 'epoch': 5.0}
Subset Accuracy: 0.1583
Hamming Loss: 0.2635
Micro Precision: 0.6873
Micro Recall: 0.5309
Micro F1-Score: 0.5990
Precision per label: [0.8        0.7804878  0.60377358 0.71111111 0.66666667 0.61904762
 0.625      0.72916667]
Recall per label: [0.34285714 0.82051282 0.64       0.5        0.23809524 0.56521739
 0.47619048 0.59322034]
F1-Score per label: [0.48       0.8        0.62135922 0.58715596 0.35087719 0.59090909
 0.54054054 0.65420561]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(7), np.int64(8)}


/tmp/ipykernel_4025847/525870965.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [195]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lara.hassan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [200]:
import os

scorer_script = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_12/camelbert_finetuned_epochs_5_eval_f1_0.7799_greater_threshold_0.3/-home-lara.hassan-Documents-Cross-Country-Dialectal-Arabic-Identification-exp_9-camelbert_finetuned_epochs_2_eval_f1_0.7812_greater_threshold_0.3-experiment-12_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 69.19 %
MACRO AVERAGE RECALL SCORE: 52.20 %
MACRO AVERAGE F1-SCORE: 57.81 %
MACRO AVERAGE ACCURACY: 73.65 %



In [201]:
trainer.predict(["الله اكبر"])

(array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 array([[0.9784671 , 0.98201376, 0.97339284, 0.97706646, 0.9714559 ,
         0.98421544, 0.98141783, 0.9826909 , 0.9749646 , 0.98521465,
         0.9819447 , 0.9868787 , 0.97414124, 0.9840936 , 0.98228765,
         0.97850823, 0.9845754 , 0.987375  ]], dtype=float32))

In [202]:
trainer.predict(["انا مصرى ياسطااااا"])

(array([[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]]),
 array([[0.02100437, 0.56673807, 0.96826136, 0.6016521 , 0.6660625 ,
         0.7235049 , 0.4344013 , 0.5190795 , 0.01851105, 0.41750893,
         0.5469499 , 0.5705108 , 0.7168137 , 0.7305783 , 0.40751025,
         0.01428194, 0.30002728, 0.4107563 ]], dtype=float32))

In [203]:
trainer.predict(['بدى فتوش و بدك ثومية'])

(array([[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0]]),
 array([[0.0186178 , 0.00944908, 0.05379965, 0.11636177, 0.87768656,
         0.0102486 , 0.91181   , 0.02546808, 0.01876109, 0.00818791,
         0.8321002 , 0.00912564, 0.02909076, 0.01317195, 0.8830344 ,
         0.01879707, 0.00575254, 0.01016966]], dtype=float32))

In [208]:
trainer.predict([' خى ما قصرت'])

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]),
 array([[0.13660839, 0.01159736, 0.08344117, 0.15241033, 0.03704717,
         0.02442309, 0.02052779, 0.19329959, 0.10212548, 0.01141961,
         0.0226292 , 0.01036816, 0.07504072, 0.01890545, 0.02465686,
         0.20323272, 0.00656425, 0.0113756 ]], dtype=float32))

In [209]:
trainer.predict(['الحمد لله يا زلمة'])

(array([[0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0]]),
 array([[0.0082516 , 0.78727776, 0.9410069 , 0.87997437, 0.9512329 ,
         0.84376645, 0.8782099 , 0.6583077 , 0.00854433, 0.36274344,
         0.9314625 , 0.71452844, 0.9380106 , 0.5837373 , 0.88602656,
         0.00919655, 0.32562777, 0.27338037]], dtype=float32))

In [210]:
trainer.predict(['الحمد لله  '])

(array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 array([[0.97029626, 0.98527133, 0.9717254 , 0.9797453 , 0.9780936 ,
         0.9874236 , 0.9840936 , 0.9809491 , 0.9636434 , 0.9873262 ,
         0.98481095, 0.98926485, 0.97879386, 0.9832145 , 0.98486924,
         0.9704086 , 0.98646784, 0.98844737]], dtype=float32))

In [211]:
trainer.predict(["كل زول ليه الزول بتاعه"])

(array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]]),
 array([[0.0349467 , 0.03507867, 0.8701566 , 0.22695492, 0.07303239,
         0.05964694, 0.03890198, 0.17497347, 0.03704717, 0.0193451 ,
         0.06176271, 0.02640552, 0.09704755, 0.4174496 , 0.03725677,
         0.02992975, 0.01802074, 0.03015741]], dtype=float32))

### EXPERIMENT 13

In [71]:
# CHANGE DATA TO BE MORE BALANCED
import pandas as pd
from sklearn.utils import shuffle, resample

directory = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/"
dataset_path = directory + "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv"
output_path  = directory + "SORTED_multilabel_dataset.csv"
df = pd.read_csv(dataset_path)
label_columns = df.columns.difference(['tweet'])
df['binary_label'] = df[label_columns].astype(str).agg(''.join, axis=1)
sorted_df = df.sort_values(by='binary_label').reset_index(drop=True)
sorted_df.to_csv(output_path, index=False)

In [72]:
label_columns = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait', 'Lebanon', 
                     'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar', 'Saudi_Arabia', 
                     'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

In [73]:

def get_class_insight(balanced_df):
    
    counts = {i: (balanced_df[label_columns].sum(axis=1) == i).sum() for i in range(19)}

    print("Counts of rows by number of active labels:")
    for count, value in counts.items():
        print(f"Class with {count} active labels: {value}")

    label_counts_by_class = {}

    
    for num_classes in range(1, len(label_columns) + 1):
        subset = balanced_df[balanced_df[label_columns].sum(axis=1) == num_classes]
        
        label_counts = subset[label_columns].sum()
        
        label_counts_by_class[num_classes] = label_counts.to_dict()

    for num_classes, counts in label_counts_by_class.items():
        print(f"\nFor class group with {num_classes} active labels:")
        for label, count in counts.items():
            print(f"  {label}: {count} occurrences")


In [77]:


def split_balanced_threshold(df, label_columns, threshold=750, uni_label=True):
    balanced_df = pd.DataFrame()
    for num_classes in range(1, len(label_columns) + 1):
        
        subset = df[df[label_columns].sum(axis=1) == num_classes]

        if num_classes == 1 and uni_label:
            balanced_subset = pd.DataFrame()
            
            
            for label in label_columns:
                label_rows = subset[subset[label] == 1]
                
                if len(label_rows) > threshold//15:
                    label_rows = resample(label_rows, n_samples=threshold//15, random_state=42, replace=False)
                elif len(label_rows) < threshold//15:
                    label_rows = resample(label_rows, n_samples=threshold//15, random_state=42, replace=True)
                
               
                balanced_subset = pd.concat([balanced_subset, label_rows], ignore_index=True)
            
        
            balanced_df = pd.concat([balanced_df, balanced_subset], ignore_index=True)
        
        else:
            if len(subset) > threshold:
                subset = shuffle(subset).head(threshold)
            balanced_df = pd.concat([balanced_df, subset], ignore_index=True)

    return balanced_df


In [78]:
def modify_dataset_for_active_labels(df):                     
    mask = df[label_columns].sum(axis=1).isin([16, 17])
    df.loc[mask, label_columns] = 1  
    return df


In [79]:
df = pd.read_csv(output_path)
df.head()

,id,tweet,Algeria,Bahrain,Egypt,Iraq,Jordan,Kuwait,Lebanon,Libya,...,Palestine,Qatar,Saudi_Arabia,Sudan,Syria,Tunisia,UAE,Yemen,Computed,binary_label
0,10012,تقريبا كلام السيسي بدل يتكلم عربي بيتكلم انجليزي,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,no,00no000000000000000110012
1,10032,وای ایشالا که حالش خوبه,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,no,00no000000000000000110032
2,10041,اعجبني فيديو علي قوات مدعومه اماراتيا تابعه لنجل,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,no,00no000000000000000110041
3,10064,حداقل دیگه حسرت اینکه چرا نگفتم رو نداری,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,no,00no000000000000000110064
4,10087,مرعبه كلمه سامحني لان اغلب الاوقات يكون بعدها ...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,no,00no000000000000000110087


In [80]:
get_class_insight(df)

Counts of rows by number of active labels:
Class with 0 active labels: 0
Class with 1 active labels: 41837
Class with 2 active labels: 3220
Class with 3 active labels: 2329
Class with 4 active labels: 2580
Class with 5 active labels: 1416
Class with 6 active labels: 1709
Class with 7 active labels: 1794
Class with 8 active labels: 1108
Class with 9 active labels: 536
Class with 10 active labels: 168
Class with 11 active labels: 137
Class with 12 active labels: 334
Class with 13 active labels: 273
Class with 14 active labels: 256
Class with 15 active labels: 532
Class with 16 active labels: 11
Class with 17 active labels: 26
Class with 18 active labels: 502

For class group with 1 active labels:
  Algeria: 2712 occurrences
  Bahrain: 417 occurrences
  Egypt: 10609 occurrences
  Iraq: 6026 occurrences
  Jordan: 762 occurrences
  Kuwait: 672 occurrences
  Lebanon: 1245 occurrences
  Libya: 2567 occurrences
  Morocco: 2213 occurrences
  Oman: 2077 occurrences
  Palestine: 794 occurrences
 

In [81]:
df = modify_dataset_for_active_labels(df)

In [82]:
get_class_insight(df)

Counts of rows by number of active labels:
Class with 0 active labels: 0
Class with 1 active labels: 41837
Class with 2 active labels: 3220
Class with 3 active labels: 2329
Class with 4 active labels: 2580
Class with 5 active labels: 1416
Class with 6 active labels: 1709
Class with 7 active labels: 1794
Class with 8 active labels: 1108
Class with 9 active labels: 536
Class with 10 active labels: 168
Class with 11 active labels: 137
Class with 12 active labels: 334
Class with 13 active labels: 273
Class with 14 active labels: 256
Class with 15 active labels: 532
Class with 16 active labels: 0
Class with 17 active labels: 0
Class with 18 active labels: 539

For class group with 1 active labels:
  Algeria: 2712 occurrences
  Bahrain: 417 occurrences
  Egypt: 10609 occurrences
  Iraq: 6026 occurrences
  Jordan: 762 occurrences
  Kuwait: 672 occurrences
  Lebanon: 1245 occurrences
  Libya: 2567 occurrences
  Morocco: 2213 occurrences
  Oman: 2077 occurrences
  Palestine: 794 occurrences
  Q

In [84]:
df = split_balanced_threshold(df, label_columns, threshold=500, uni_label=True)

In [85]:
get_class_insight(df)

Counts of rows by number of active labels:
Class with 0 active labels: 0
Class with 1 active labels: 594
Class with 2 active labels: 500
Class with 3 active labels: 500
Class with 4 active labels: 500
Class with 5 active labels: 500
Class with 6 active labels: 500
Class with 7 active labels: 500
Class with 8 active labels: 500
Class with 9 active labels: 500
Class with 10 active labels: 168
Class with 11 active labels: 137
Class with 12 active labels: 334
Class with 13 active labels: 273
Class with 14 active labels: 256
Class with 15 active labels: 500
Class with 16 active labels: 0
Class with 17 active labels: 0
Class with 18 active labels: 500

For class group with 1 active labels:
  Algeria: 33 occurrences
  Bahrain: 33 occurrences
  Egypt: 33 occurrences
  Iraq: 33 occurrences
  Jordan: 33 occurrences
  Kuwait: 33 occurrences
  Lebanon: 33 occurrences
  Libya: 33 occurrences
  Morocco: 33 occurrences
  Oman: 33 occurrences
  Palestine: 33 occurrences
  Qatar: 33 occurrences
  Saudi

In [86]:
df.to_csv(directory + " BALANCED_NEW_DATASET_500.csv", index=False)

In [69]:
dataset_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/ BALANCED_NEW_DATASET_750.csv"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_8/camelbert_finetuned_epochs_2_eval_f1_0.7694_greater_threshold_0.3",
    labels=labels,
    threshold=0.3,
    exp_num=13
)
trainer.train(
    num_train_epochs=2,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)

/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                                 
 50%|█████     | 374/748 [00:32<00:27, 13.65it/s]

{'eval_loss': 0.4429030120372772, 'eval_f1': 0.7262548011197187, 'eval_roc_auc': 0.7910349145094046, 'eval_accuracy': 0.15060240963855423, 'eval_runtime': 0.7421, 'eval_samples_per_second': 1342.187, 'eval_steps_per_second': 56.598, 'epoch': 1.0}


 67%|██████▋   | 501/748 [00:43<00:19, 12.81it/s]

{'loss': 0.4215, 'grad_norm': 4.750375270843506, 'learning_rate': 5e-05, 'epoch': 1.34}


                                                 
100%|██████████| 748/748 [01:05<00:00, 13.22it/s]

{'eval_loss': 0.4512268900871277, 'eval_f1': 0.7295721119192713, 'eval_roc_auc': 0.7940113323604024, 'eval_accuracy': 0.16967871485943775, 'eval_runtime': 0.7301, 'eval_samples_per_second': 1364.209, 'eval_steps_per_second': 57.527, 'epoch': 2.0}


100%|██████████| 748/748 [01:08<00:00, 10.96it/s]


{'train_runtime': 68.2553, 'train_samples_per_second': 262.397, 'train_steps_per_second': 10.959, 'train_loss': 0.4062513361640155, 'epoch': 2.0}
Subset Accuracy: 0.1917
Hamming Loss: 0.2740
Micro Precision: 0.6768
Micro Recall: 0.5000
Micro F1-Score: 0.5751
Precision per label: [0.8        0.89655172 0.58823529 0.66666667 0.66666667 0.55813953
 0.64705882 0.73809524]
Recall per label: [0.34285714 0.66666667 0.6        0.5        0.28571429 0.52173913
 0.52380952 0.52542373]
F1-Score per label: [0.48       0.76470588 0.59405941 0.57142857 0.4        0.53932584
 0.57894737 0.61386139]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(8)}


/tmp/ipykernel_1992904/525870965.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [70]:
import os

scorer_script = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_13/camelbert_finetuned_epochs_2_eval_f1_0.7296_greater_threshold_0.3/-home-lara.hassan-Documents-Cross-Country-Dialectal-Arabic-Identification-exp_8-camelbert_finetuned_epochs_2_eval_f1_0.7694_greater_threshold_0.3-experiment-13_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 69.52 %
MACRO AVERAGE RECALL SCORE: 49.58 %
MACRO AVERAGE F1-SCORE: 56.78 %
MACRO AVERAGE ACCURACY: 72.60 %



### EXPERIMENT 14

In [88]:
dataset_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/ BALANCED_NEW_DATASET_500.csv"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_8/camelbert_finetuned_epochs_2_eval_f1_0.7694_greater_threshold_0.3",
    labels=labels,
    threshold=0.3,
    exp_num=14
)
trainer.train(
    num_train_epochs=5,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)

/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 20%|██        | 254/1270 [00:20<01:17, 13.17it/s]

{'eval_loss': 0.41958171129226685, 'eval_f1': 0.7857666313818092, 'eval_roc_auc': 0.7992212978015347, 'eval_accuracy': 0.17429837518463812, 'eval_runtime': 0.514, 'eval_samples_per_second': 1317.176, 'eval_steps_per_second': 56.423, 'epoch': 1.0}


 39%|███▉      | 501/1270 [00:41<00:58, 13.14it/s]

{'loss': 0.3978, 'grad_norm': 2.486278533935547, 'learning_rate': 5e-05, 'epoch': 1.97}


 40%|████      | 508/1270 [00:42<00:58, 13.04it/s]

...

100%|██████████| 1270/1270 [01:52<00:00, 13.10it/s]

{'eval_loss': 0.4963809847831726, 'eval_f1': 0.7906378600823045, 'eval_roc_auc': 0.8081659234473502, 'eval_accuracy': 0.17725258493353027, 'eval_runtime': 0.5111, 'eval_samples_per_second': 1324.562, 'eval_steps_per_second': 56.739, 'epoch': 5.0}


100%|██████████| 1270/1270 [01:55<00:00, 10.99it/s]


{'train_runtime': 115.591, 'train_samples_per_second': 263.213, 'train_steps_per_second': 10.987, 'train_loss': 0.31099345590185934, 'epoch': 5.0}
Subset Accuracy: 0.1583
Hamming Loss: 0.2844
Micro Precision: 0.6554
Micro Recall: 0.4916
Micro F1-Score: 0.5618
Precision per label: [0.81818182 0.90322581 0.50909091 0.63265306 0.55       0.55813953
 0.61538462 0.8       ]
Recall per label: [0.25714286 0.71794872 0.56       0.484375   0.26190476 0.52173913
 0.38095238 0.61016949]
F1-Score per label: [0.39130435 0.8        0.53333333 0.54867257 0.35483871 0.53932584
 0.47058824 0.69230769]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(8)}


/tmp/ipykernel_1992904/525870965.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [89]:
import os

scorer_script = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_14/camelbert_finetuned_epochs_5_eval_f1_0.7906_greater_threshold_0.3/-home-lara.hassan-Documents-Cross-Country-Dialectal-Arabic-Identification-exp_8-camelbert_finetuned_epochs_2_eval_f1_0.7694_greater_threshold_0.3-experiment-14_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 67.33 %
MACRO AVERAGE RECALL SCORE: 47.43 %
MACRO AVERAGE F1-SCORE: 54.13 %
MACRO AVERAGE ACCURACY: 71.56 %



### EXPERIMENT 15

In [90]:
dataset_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/ BALANCED_NEW_DATASET_500.csv"
dev_path = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_8/camelbert_finetuned_epochs_2_eval_f1_0.7694_greater_threshold_0.3",
    labels=labels,
    threshold=0.3,
    exp_num=14
)
trainer.train(
    num_train_epochs=20,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)

/home/lara.hassan/.conda/envs/myenv/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  5%|▌         | 254/5080 [00:20<06:05, 13.19it/s]

{'eval_loss': 0.41958171129226685, 'eval_f1': 0.7857666313818092, 'eval_roc_auc': 0.7992212978015347, 'eval_accuracy': 0.17429837518463812, 'eval_runtime': 0.5048, 'eval_samples_per_second': 1340.993, 'eval_steps_per_second': 57.443, 'epoch': 1.0}


 10%|▉         | 502/5080 [00:41<05:50, 13.05it/s]

{'loss': 0.3978, 'grad_norm': 2.486278533935547, 'learning_rate': 5e-05, 'epoch': 1.97}


 10%|█         | 508/5080 [00:42<05:39, 13.46it/s]

...

100%|██████████| 5080/5080 [07:35<00:00, 13.52it/s]

{'eval_loss': 0.799960196018219, 'eval_f1': 0.7806718160589187, 'eval_roc_auc': 0.8022875573263056, 'eval_accuracy': 0.155096011816839, 'eval_runtime': 0.5184, 'eval_samples_per_second': 1305.893, 'eval_steps_per_second': 55.939, 'epoch': 20.0}


100%|██████████| 5080/5080 [07:41<00:00, 11.01it/s]


{'train_runtime': 461.3046, 'train_samples_per_second': 263.817, 'train_steps_per_second': 11.012, 'train_loss': 0.14231886192569582, 'epoch': 20.0}
Subset Accuracy: 0.1333
Hamming Loss: 0.2969
Micro Precision: 0.6473
Micro Recall: 0.4382
Micro F1-Score: 0.5226
Precision per label: [0.76923077 0.88       0.5        0.65853659 0.46153846 0.61764706
 0.53333333 0.74      ]
Recall per label: [0.28571429 0.56410256 0.5        0.421875   0.14285714 0.45652174
 0.38095238 0.62711864]
F1-Score per label: [0.41666667 0.6875     0.5        0.51428571 0.21818182 0.525
 0.44444444 0.67889908]
{np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4), np.int64(5), np.int64(6), np.int64(8)}


/tmp/ipykernel_1992904/525870965.py:81: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [92]:
import os

scorer_script = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/lara.hassan/Documents/Cross-Country-Dialectal-Arabic-Identification/exp_14/camelbert_finetuned_epochs_20_eval_f1_0.7881_greater_threshold_0.3/-home-lara.hassan-Documents-Cross-Country-Dialectal-Arabic-Identification-exp_8-camelbert_finetuned_epochs_2_eval_f1_0.7694_greater_threshold_0.3-experiment-14_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 64.50 %
MACRO AVERAGE RECALL SCORE: 42.24 %
MACRO AVERAGE F1-SCORE: 49.81 %
MACRO AVERAGE ACCURACY: 70.31 %



In [93]:
trainer.predict(["الله اكبر"])

(array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 array([[0.9507779 , 0.98521465, 0.97288233, 0.97754294, 0.98201376,
         0.9885804 , 0.98475236, 0.96628344, 0.9546474 , 0.98197925,
         0.9845754 , 0.9857729 , 0.9827572 , 0.97788346, 0.98521465,
         0.94539934, 0.98340684, 0.9881765 ]], dtype=float32))

In [94]:
trainer.predict(["انا مصرى ياسطااااا"])

(array([[0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1]]),
 array([[0.00941259, 0.624179  , 0.7916601 , 0.4977589 , 0.6311397 ,
         0.73278314, 0.33133605, 0.26999873, 0.00754792, 0.4181622 ,
         0.47498128, 0.66093874, 0.86025184, 0.3869372 , 0.33654907,
         0.00871138, 0.32637876, 0.47081262]], dtype=float32))

## Experiment 16 Mekky

In [6]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
# dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dataset_path = '/home/ali.mekky/Documents/NLP/Project/Cross-Country-Dialectal-Arabic-Identification/balanced_multilabel_dataset_500.csv'
dev_path = "/home/ali.mekky/Documents/NLP/Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="/home/ali.mekky/Documents/NLP/Project/Cross-Country-Dialectal-Arabic-Identification/exp_16/marbert_finetuned_epochs_2_eval_f1_0.7607_greater_threshold_0.3",
    labels=labels,
    threshold=0.3,
    exp_num=17
)
trainer.train(
    num_train_epochs=10,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24
)
trainer.evaluate(dev_path=dev_path)

/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                                  
 10%|█         | 252/2520 [00:22<03:11, 11.85it/s]

{'eval_loss': 0.467267781496048, 'eval_f1': 0.7741051549922932, 'eval_roc_auc': 0.7878222018813346, 'eval_accuracy': 0.16542473919523099, 'eval_runtime': 0.5593, 'eval_samples_per_second': 1199.706, 'eval_steps_per_second': 50.062, 'epoch': 1.0}


 20%|█▉        | 502/2520 [00:49<03:01, 11.11it/s]

{'loss': 0.4168, 'grad_norm': 6.076703071594238, 'learning_rate': 5e-05, 'epoch': 1.98}


                                                  
 20%|██        | 504/2520 [00:49<02:55, 11.50it/s]

...

                                                   
100%|██████████| 2520/2520 [04:25<00:00, 11.81it/s]

{'eval_loss': 0.6510425209999084, 'eval_f1': 0.7713090465400035, 'eval_roc_auc': 0.7870727950749379, 'eval_accuracy': 0.13710879284649777, 'eval_runtime': 0.5029, 'eval_samples_per_second': 1334.228, 'eval_steps_per_second': 55.676, 'epoch': 10.0}


100%|██████████| 2520/2520 [04:30<00:00,  9.31it/s]


{'train_runtime': 270.5929, 'train_samples_per_second': 222.992, 'train_steps_per_second': 9.313, 'train_loss': 0.24082989210174197, 'epoch': 10.0}
Subset Accuracy: 0.1667
Hamming Loss: 0.2510
Micro Precision: 0.7061
Micro Recall: 0.5534
Micro F1-Score: 0.6205
Precision per label: [0.66666667 0.90909091 0.6        0.72222222 0.77777778 0.59259259
 0.6        0.84444444]
Recall per label: [0.17142857 0.76923077 0.78       0.609375   0.16666667 0.69565217
 0.28571429 0.6440678 ]
F1-Score per label: [0.27272727 0.83333333 0.67826087 0.66101695 0.2745098  0.64
 0.38709677 0.73076923]
{0, 1, 2, 3, 4, 5, 6}


/tmp/ipykernel_2559847/247078281.py:82: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [8]:
import os

scorer_script = "/home/ali.mekky/Documents/NLP/Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/ali.mekky/Documents/NLP/Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/ali.mekky/Documents/NLP/Project/Cross-Country-Dialectal-Arabic-Identification/exp_17/marbert_finetuned_epochs_10_eval_f1_0.7785_greater_threshold_0.3/-home-ali.mekky-Documents-NLP-Project-Cross-Country-Dialectal-Arabic-Identification-exp_16-marbert_finetuned_epochs_2_eval_f1_0.7607_greater_threshold_0.3-experiment-17_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 71.41 %
MACRO AVERAGE RECALL SCORE: 51.53 %
MACRO AVERAGE F1-SCORE: 55.97 %
MACRO AVERAGE ACCURACY: 74.90 %



## EXP 18

In [9]:
import pandas as pd
from sklearn.utils import shuffle

dataset_path = '/home/ali.mekky/Documents/NLP/Project/Cross-Country-Dialectal-Arabic-Identification/lr_binary_classifiers/annotated_multi_label_logisitc_regression.csv'
df = pd.read_csv(dataset_path)


label_columns = df.columns[2:-1]  
df[label_columns] = df[label_columns].astype(int) 

threshold = 500  
balanced_df = pd.DataFrame()


for num_classes in range(1, len(label_columns) + 1):
    subset = df[df[label_columns].sum(axis=1) == num_classes]  
    
    if len(subset) > threshold:
        subset = shuffle(subset).head(threshold)
    
    balanced_df = pd.concat([balanced_df, subset], ignore_index=True)

balanced_df = shuffle(balanced_df).reset_index(drop=True)
balanced_df.to_csv('balanced_multilabel_dataset_lr_' + str(threshold) + '.csv', index=False)

print("Balanced dataset created and saved as 'balanced_multilabel_dataset_500.csv'")

label_columns = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait', 'Lebanon', 
                 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar', 'Saudi_Arabia', 
                 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

counts = {}

for i in range(19):
    counts[i] = (balanced_df[label_columns].sum(axis=1) == i).sum()

Balanced dataset created and saved as 'balanced_multilabel_dataset_500.csv'


In [8]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = '/home/ali.mekky/Documents/NLP/Project/Cross-Country-Dialectal-Arabic-Identification/lr_binary_classifiers/annotated_multi_label_logisitc_regression.csv'
dev_path = "/home/ali.mekky/Documents/NLP/Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-ca",
    labels=labels,
    threshold=0.3,
    exp_num=30
)
trainer.train(
    num_train_epochs=10,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-ca and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▏         | 503/22040 [00:34<21:18, 16.85it/s] 

{'loss': 0.5979, 'grad_norm': 1.3358558416366577, 'learning_rate': 5e-05, 'epoch': 0.23}


  5%|▍         | 1003/22040 [01:04<20:59, 16.70it/s]

{'loss': 0.5129, 'grad_norm': 1.0376207828521729, 'learning_rate': 4.883936861652739e-05, 'epoch': 0.45}


  7%|▋         | 1503/22040 [01:34<20:34, 16.64it/s]

...

                                                     
100%|██████████| 22040/22040 [21:19<00:00, 18.12it/s]

{'eval_loss': 0.2849634289741516, 'eval_f1': 0.818281100958016, 'eval_roc_auc': 0.8725174125949922, 'eval_accuracy': 0.18104475072315807, 'eval_runtime': 4.5205, 'eval_samples_per_second': 1300.087, 'eval_steps_per_second': 54.198, 'epoch': 10.0}


100%|██████████| 22040/22040 [21:21<00:00, 17.20it/s]


{'train_runtime': 1281.7072, 'train_samples_per_second': 412.661, 'train_steps_per_second': 17.196, 'train_loss': 0.26628883518887086, 'epoch': 10.0}
Subset Accuracy: 0.1000
Hamming Loss: 0.3333
Micro Precision: 0.5462
Micro Recall: 0.5983
Micro F1-Score: 0.5710
Precision per label: [0.51162791 0.63265306 0.53846154 0.57692308 0.73076923 0.48484848
 0.38095238 0.58333333]
Recall per label: [0.62857143 0.79487179 0.56       0.46875    0.45238095 0.69565217
 0.76190476 0.59322034]
F1-Score per label: [0.56410256 0.70454545 0.54901961 0.51724138 0.55882353 0.57142857
 0.50793651 0.58823529]
{0, 1, 2, 3, 4, 5, 6, 7, 8}


/tmp/ipykernel_4102662/3917706724.py:77: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [12]:
import os

scorer_script = "/home/ali.mekky/Documents/NLP/Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/ali.mekky/Documents/NLP/Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/ali.mekky/Documents/NLP/Project/Cross-Country-Dialectal-Arabic-Identification/exp_18/marbert_finetuned_epochs_10_eval_f1_0.8353_greater_threshold_0.3/CAMeL-Lab-bert-base-arabic-camelbert-ca-experiment-18_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 46.80 %
MACRO AVERAGE RECALL SCORE: 64.37 %
MACRO AVERAGE F1-SCORE: 52.38 %
MACRO AVERAGE ACCURACY: 58.44 %



In [13]:
trainer.predict(["الله اكبر"])

(array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 array([[0.99621993, 0.99293363, 0.9789554 , 0.9881765 , 0.99378294,
         0.99346113, 0.98922324, 0.99135584, 0.9954261 , 0.9932811 ,
         0.994292  , 0.9939731 , 0.994089  , 0.9908035 , 0.99158704,
         0.99317604, 0.9935369 , 0.9952632 ]], dtype=float32),
 0.007803493075900558)

In [17]:
trainer.predict(["إيه يا عم الجو حر موت النهارده، لازم نشرب حاجة ساقعة"])

(array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]),
 array([[0.00602695, 0.01210039, 0.02551661, 0.13398075, 0.02532306,
         0.01596765, 0.12995382, 0.0085113 , 0.00219124, 0.01200735,
         0.04689926, 0.00372171, 0.01778039, 0.00564191, 0.43236297,
         0.0102883 , 0.01764446, 0.0124797 ]], dtype=float32),
 0.9489779008759393)

## Experiment 19

In [22]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = '/home/ali.mekky/Documents/NLP/Project/Cross-Country-Dialectal-Arabic-Identification/lr_binary_classifiers/annotated_multi_label_logisitc_regression.csv'
dev_path = "/home/ali.mekky/Documents/NLP/Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="UBC-NLP/MARBERT",
    labels=labels,
    threshold=0.3,
    exp_num=18
)
trainer.train(
    num_train_epochs=10,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                                  
  0%|          | 17/22040 [3:31:48<28:04, 13.07it/s]

{'loss': 0.5377, 'grad_norm': 2.3052988052368164, 'learning_rate': 5e-05, 'epoch': 0.23}


                                                    
  0%|          | 17/22040 [3:32:33<28:04, 13.07it/s]

{'loss': 0.4135, 'grad_norm': 1.8364241123199463, 'learning_rate': 4.883936861652739e-05, 'epoch': 0.45}


                                                    
  0%|          | 17/22040 [3:33:18<28:04, 13.07it/s]

...












































                                                    
                                                  
  0%|          | 17/22040 [4:04:18<28:04, 13.07it/s]


{'eval_loss': 0.3677639365196228, 'eval_f1': 0.8469693880523111, 'eval_roc_auc': 0.8870306658177636, 'eval_accuracy': 0.2285179513357155, 'eval_runtime': 4.508, 'eval_samples_per_second': 1303.674, 'eval_steps_per_second': 54.347, 'epoch': 10.0}


                                                    
100%|██████████| 22040/22040 [33:18<00:00, 11.03it/s]


{'train_runtime': 1998.5777, 'train_samples_per_second': 264.643, 'train_steps_per_second': 11.028, 'train_loss': 0.10601788969835547, 'epoch': 10.0}
Subset Accuracy: 0.0917
Hamming Loss: 0.3302
Micro Precision: 0.5499
Micro Recall: 0.6039
Micro F1-Score: 0.5756
Precision per label: [0.54901961 0.68181818 0.54       0.58695652 0.63333333 0.50724638
 0.34693878 0.61538462]
Recall per label: [0.8        0.76923077 0.54       0.421875   0.45238095 0.76086957
 0.80952381 0.54237288]
F1-Score per label: [0.65116279 0.72289157 0.54       0.49090909 0.52777778 0.60869565
 0.48571429 0.57657658]
{0, 1, 2, 3, 4, 5, 6, 7, 8}


/tmp/ipykernel_2559847/4032790609.py:82: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [1]:
import os

scorer_script = "/home/ali.mekky/Documents/NLP/Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "/home/ali.mekky/Documents/NLP/Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "/home/ali.mekky/Documents/NLP/Project/Cross-Country-Dialectal-Arabic-Identification/exp_18/marbert_finetuned_epochs_10_eval_f1_0.8516_greater_threshold_0.3/UBC-NLP-MARBERT-experiment-18_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"


OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 55.76 %
MACRO AVERAGE RECALL SCORE: 63.70 %
MACRO AVERAGE F1-SCORE: 57.55 %
MACRO AVERAGE ACCURACY: 66.98 %



## Experiment 20

balancing the dataset produced by the logistic regression and finetuning marbert

In [ ]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
# dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dataset_path = 'Project/Cross-Country-Dialectal-Arabic-Identification/balanced_multilabel_dataset_lr_500.csv'
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="Project/Cross-Country-Dialectal-Arabic-Identification/exp_20/marbert_finetuned_epochs_10_eval_f1_0.8621_greater_threshold_0.3",
    labels=labels,
    threshold=0.3,
    exp_num=21
)
trainer.train(
    num_train_epochs=20,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

In [1]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_20/marbert_finetuned_epochs_20_eval_f1_0.8577_greater_threshold_0.3/-home-ali.mekky-Documents-NLP-Project-Cross-Country-Dialectal-Arabic-Identification-exp_20-marbert_finetuned_epochs_10_eval_f1_0.8621_greater_threshold_0.3-experiment-20_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"


OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 53.12 %
MACRO AVERAGE RECALL SCORE: 64.70 %
MACRO AVERAGE F1-SCORE: 55.99 %
MACRO AVERAGE ACCURACY: 64.48 %



## EXP 21

balancing the dataset produced by the logistic regression and finetuning camelbert

In [3]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = 'Project/Cross-Country-Dialectal-Arabic-Identification/balanced_multilabel_dataset_lr_500.csv'
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-ca",
    labels=labels,
    threshold=0.3,
    exp_num=21
)
trainer.train(
    num_train_epochs=20,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-ca and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
                                                   
  5%|▌         | 319/6380 [00:30<07:29, 13.48it/s]

{'eval_loss': 0.5511866211891174, 'eval_f1': 0.7437626750931472, 'eval_roc_auc': 0.6177656031477913, 'eval_accuracy': 0.07529411764705882, 'eval_runtime': 0.7482, 'eval_samples_per_second': 1136.014, 'eval_steps_per_second': 48.114, 'epoch': 1.0}


  8%|▊         | 501/6380 [00:47<07:57, 12.31it/s]

{'loss': 0.5907, 'grad_norm': 6.361403942108154, 'learning_rate': 4.96e-05, 'epoch': 1.57}


                                                  
 10%|█         | 638/6380 [00:59<07:41, 12.43it/s]

...

{'eval_loss': 0.5265424847602844, 'eval_f1': 0.8449036452286974, 'eval_roc_auc': 0.8205344921247426, 'eval_accuracy': 0.14941176470588236, 'eval_runtime': 0.5301, 'eval_samples_per_second': 1603.392, 'eval_steps_per_second': 67.908, 'epoch': 20.0}


100%|██████████| 6380/6380 [09:45<00:00, 10.91it/s]


{'train_runtime': 585.0119, 'train_samples_per_second': 261.533, 'train_steps_per_second': 10.906, 'train_loss': 0.19146193889988627, 'epoch': 20.0}


/tmp/ipykernel_845921/3956158625.py:82: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


Subset Accuracy: 0.1000
Hamming Loss: 0.3635
Micro Precision: 0.5081
Micro Recall: 0.6152
Micro F1-Score: 0.5565
Precision per label: [0.42105263 0.50909091 0.49152542 0.62745098 0.65517241 0.53030303
 0.29787234 0.56716418]
Recall per label: [0.68571429 0.71794872 0.58       0.5        0.45238095 0.76086957
 0.66666667 0.6440678 ]
F1-Score per label: [0.52173913 0.59574468 0.53211009 0.55652174 0.53521127 0.625
 0.41176471 0.6031746 ]
{0, 1, 2, 3, 4, 5, 6, 7, 8}


In [5]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_21/marbert_finetuned_epochs_20_eval_f1_0.8457_greater_threshold_0.3/CAMeL-Lab-bert-base-arabic-camelbert-ca-experiment-21_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 51.25 %
MACRO AVERAGE RECALL SCORE: 62.60 %
MACRO AVERAGE F1-SCORE: 54.77 %
MACRO AVERAGE ACCURACY: 63.65 %



## EXP 22

freezing lower layers and adding dropout

In [9]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
# dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dataset_path = 'Project/Cross-Country-Dialectal-Arabic-Identification/lr_binary_classifiers/annotated_multi_label_logisitc_regression.csv'
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="UBC-NLP/MARBERT",
    labels=labels,
    threshold=0.3,
    exp_num=22
)
trainer.train(
    num_train_epochs=10,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

  1%|          | 119/22040 [17:08<52:38:49,  8.65s/it]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▏         | 502/22040 [00:34<25:11, 14.25it/s]

{'loss': 0.4872, 'grad_norm': 2.7567391395568848, 'learning_rate': 5e-05, 'epoch': 0.23}


  5%|▍         | 1002/22040 [01:08<25:13, 13.90it/s]

{'loss': 0.4106, 'grad_norm': 2.746913194656372, 'learning_rate': 5e-05, 'epoch': 0.45}


  7%|▋         | 1502/22040 [01:43<23:29, 14.57it/s]

...

 50%|█████     | 11020/22040 [12:42<11:44, 15.63it/s]

{'eval_loss': 0.31265273690223694, 'eval_f1': 0.8298505777940823, 'eval_roc_auc': 0.8786760389194355, 'eval_accuracy': 0.18291645397311554, 'eval_runtime': 4.5224, 'eval_samples_per_second': 1299.54, 'eval_steps_per_second': 54.175, 'epoch': 5.0}


 50%|█████     | 11020/22040 [12:46<12:46, 14.37it/s]


{'train_runtime': 766.6979, 'train_samples_per_second': 689.854, 'train_steps_per_second': 28.747, 'train_loss': 0.21260312045983523, 'epoch': 5.0}
Subset Accuracy: 0.1000
Hamming Loss: 0.3156
Micro Precision: 0.5692
Micro Recall: 0.6124
Micro F1-Score: 0.5900
Precision per label: [0.54761905 0.71052632 0.54716981 0.64814815 0.73913043 0.53731343
 0.34782609 0.58333333]
Recall per label: [0.65714286 0.69230769 0.58       0.546875   0.4047619  0.7826087
 0.76190476 0.59322034]
F1-Score per label: [0.5974026  0.7012987  0.5631068  0.59322034 0.52307692 0.63716814
 0.47761194 0.58823529]
{0, 1, 2, 3, 4, 5, 6, 7, 8}


/tmp/ipykernel_875058/1135678770.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [11]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_22/marbert_finetuned_epochs_10_eval_f1_0.8341_greater_threshold_0.3/UBC-NLP-MARBERT-experiment-22_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 58.26 %
MACRO AVERAGE RECALL SCORE: 62.74 %
MACRO AVERAGE F1-SCORE: 58.51 %
MACRO AVERAGE ACCURACY: 68.44 %



## EXP 23

same as above but using camelbert

In [12]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = 'Project/Cross-Country-Dialectal-Arabic-Identification/lr_binary_classifiers/annotated_multi_label_logisitc_regression.csv'
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-ca",
    labels=labels,
    threshold=0.3,
    exp_num=23
)
trainer.train(
    num_train_epochs=10,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-ca and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  2%|▏         | 502/22040 [00:28<20:15, 17.71it/s]

{'loss': 0.5649, 'grad_norm': 1.48406183719635, 'learning_rate': 5e-05, 'epoch': 0.23}


  5%|▍         | 1002/22040 [00:57<20:23, 17.19it/s]

{'loss': 0.4982, 'grad_norm': 1.0999702215194702, 'learning_rate': 5e-05, 'epoch': 0.45}


  7%|▋         | 1502/22040 [01:26<19:48, 17.28it/s]

...

100%|██████████| 22040/22040 [22:45<00:00, 16.71it/s]

{'eval_loss': 0.3176794946193695, 'eval_f1': 0.8254703494024133, 'eval_roc_auc': 0.8735565425375277, 'eval_accuracy': 0.19499744767738642, 'eval_runtime': 4.9762, 'eval_samples_per_second': 1181.022, 'eval_steps_per_second': 49.234, 'epoch': 10.0}


100%|██████████| 22040/22040 [22:47<00:00, 16.12it/s]


{'train_runtime': 1367.1415, 'train_samples_per_second': 386.873, 'train_steps_per_second': 16.121, 'train_loss': 0.24643782885667417, 'epoch': 10.0}
Subset Accuracy: 0.1083
Hamming Loss: 0.3344
Micro Precision: 0.5482
Micro Recall: 0.5590
Micro F1-Score: 0.5535
Precision per label: [0.54545455 0.59574468 0.57777778 0.61702128 0.64       0.51612903
 0.35714286 0.56862745]
Recall per label: [0.68571429 0.71794872 0.52       0.453125   0.38095238 0.69565217
 0.71428571 0.49152542]
F1-Score per label: [0.60759494 0.65116279 0.54736842 0.52252252 0.47761194 0.59259259
 0.47619048 0.52727273]
{0, 1, 2, 3, 4, 5, 6, 7, 8}


/tmp/ipykernel_875058/1135678770.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [13]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_23/marbert_finetuned_epochs_10_eval_f1_0.8255_greater_threshold_0.3/CAMeL-Lab-bert-base-arabic-camelbert-ca-experiment-23_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 55.22 %
MACRO AVERAGE RECALL SCORE: 58.24 %
MACRO AVERAGE F1-SCORE: 55.03 %
MACRO AVERAGE ACCURACY: 66.56 %



Random baseline

In [17]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = 'Project/Cross-Country-Dialectal-Arabic-Identification/lr_binary_classifiers/annotated_multi_label_logisitc_regression.csv'
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="Project/Cross-Country-Dialectal-Arabic-Identification/exp_5/marbert_finetuned",
    labels=labels,
    threshold=0.3,
    exp_num=5
)

trainer.save_dir = f'./exp_{trainer.exp_num}'
trainer.evaluate(dev_path=dev_path)

Subset Accuracy: 0.1000
Hamming Loss: 0.6292
Micro Precision: 0.3708
Micro Recall: 1.0000
Micro F1-Score: 0.5410
Precision per label: [0.29166667 0.325      0.41666667 0.53333333 0.35       0.38333333
 0.175      0.49166667]
Recall per label: [1. 1. 1. 1. 1. 1. 1. 1.]
F1-Score per label: [0.4516129  0.49056604 0.58823529 0.69565217 0.51851852 0.55421687
 0.29787234 0.65921788]
{8}


/tmp/ipykernel_875058/1135678770.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [18]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_5/-home-ali.mekky-Documents-NLP-Project-Cross-Country-Dialectal-Arabic-Identification-exp_5-marbert_finetuned-experiment-5_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 37.08 %
MACRO AVERAGE RECALL SCORE: 100.00 %
MACRO AVERAGE F1-SCORE: 53.20 %
MACRO AVERAGE ACCURACY: 37.08 %



## exp 24

Oversampling and Undersampling to balance the dataset

In [34]:
dataset = pd.read_csv("Project/NADI2024/subtask1/multilabel/NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv")

In [35]:
dataset = dataset[dataset['Computed'] == 'yes']

In [36]:
dataset

,id,tweet,Algeria,Bahrain,Egypt,Iraq,Jordan,Kuwait,Lebanon,Libya,...,Oman,Palestine,Qatar,Saudi_Arabia,Sudan,Syria,Tunisia,UAE,Yemen,Computed
0,0,الفار العور يشوف فقط كيسي ومايشوف ماتويد,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,yes
1,1,ديني ربنا يستر,0,0,1,1,1,0,1,0,...,0,1,0,0,1,1,0,0,0,yes
2,2,اساسا نسبكم قذر ونجس بلاش تتفاخروا بنجاستكم وه...,0,0,1,1,1,0,1,0,...,0,1,0,1,0,1,0,0,1,yes
3,3,المشاعر تحتاج الي المشاعر تحتاج الي رفيق يخذل ...,0,0,1,0,1,0,1,0,...,0,1,0,0,0,0,0,0,0,yes
4,4,ني حاضرها لايف,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58742,58734,جماعه الخير المنخفض الي جاي ايام منخفض قوي ماط...,0,0,0,0,1,0,1,0,...,0,1,0,0,0,1,0,0,0,yes
58743,58735,انا بايع الكل وشاري عيونك,0,0,1,0,1,0,0,0,...,0,1,0,1,1,0,0,0,0,yes
58744,58736,USER USER USER شكلها نست يوم ترامب قال للامريك...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,yes
58745,58737,السطلات البنقو الحشيش معاكم URL ΉМĄDĄ,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,yes


In [38]:
label_columns = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait', 'Lebanon', 
                 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar', 'Saudi_Arabia', 
                 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

In [39]:
dataset['dialect_sum'] = dataset[label_columns].sum(axis=1)

In [40]:
dataset['dialect_sum'].value_counts()

dialect_sum
1     14829
2      3220
4      2580
3      2329
7      1794
6      1709
5      1416
8      1108
9       536
15      532
18      502
12      334
13      273
14      256
10      168
11      137
17       26
16       11
Name: count, dtype: int64

In [23]:
desired_samples = 1500

In [24]:
for index, row in dataset.iterrows():
    if row['dialect_sum'] in [16, 17]:
        dataset.at[index, 'dialect_sum'] = 18

        dataset.loc[index, label_columns] = row[label_columns].apply(lambda x: 1 if x == 0 else x)

print(dataset['dialect_sum'].value_counts().sort_index())

dialect_sum
1     14829
2      3220
3      2329
4      2580
5      1416
6      1709
7      1794
8      1108
9       536
10      168
11      137
12      334
13      273
14      256
15      532
18      539
Name: count, dtype: int64


In [25]:
from sklearn.utils import resample


dialect_columns = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']


rows_with_sum_1 = dataset[dataset['dialect_sum'] == 1]


balanced_rows = pd.DataFrame()


for dialect in dialect_columns:
    dialect_rows = rows_with_sum_1[rows_with_sum_1[dialect] == 1]
    
    resampled_rows = resample(dialect_rows, replace=True, 
                              n_samples=150, 
                              random_state=42)
    balanced_rows = pd.concat([balanced_rows, resampled_rows])

other_rows = dataset[dataset['dialect_sum'] != 1]
balanced_dataset = pd.concat([balanced_rows, other_rows]).reset_index(drop=True)


print("Balanced rows with dialect_sum = 1:")
print(balanced_rows[dialect_columns].sum())


Balanced rows with dialect_sum = 1:
Algeria         150
Bahrain         150
Egypt           150
Iraq            150
Jordan          150
Kuwait          150
Lebanon         150
Libya           150
Morocco         150
Oman            150
Palestine       150
Qatar           150
Saudi_Arabia    150
Sudan           150
Syria           150
Tunisia         150
UAE             150
Yemen           150
dtype: int64


In [27]:
balanced_rows

,id,tweet,Algeria,Bahrain,Egypt,Iraq,Jordan,Kuwait,Lebanon,Libya,...,Palestine,Qatar,Saudi_Arabia,Sudan,Syria,Tunisia,UAE,Yemen,Computed,dialect_sum
6805,6803,السنا بالقوارير رفقا بانفسنا المره حتي نخدش,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,yes,1
55262,55262,USER USER USER لكان نتي جزايريه واش قاعده تبعي...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,yes,1
47614,47614,USER اختي مالكي افهمي مليح باش تهدري راني علي ...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,yes,1
25429,24865,ÙÙ Ø± Ø§Ø®ÙØ§ÙØ§,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,yes,1
7008,7008,المشي طريق الامير يملاها العبيد يثير حيرتي ولا...,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47955,47955,USER حركاتك يبو يمن ياسعم ماتعرفيش تتحاكي صنعاني,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,yes,1
55817,55817,جيش اليمن ركبوه سيكل وهيكلوه فصار كالهيكل,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,yes,1
3781,3781,تكون المسافات وساءل النقل تشكل عامل مساعدا الت...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,yes,1
5075,5075,اخرجمعهفيرمضان اللهم انك عفو كريم تحب العفو فا...,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,yes,1


In [13]:
from sklearn.utils import resample
import pandas as pd

desired_samples = 1500

filtered_dataset = dataset[dataset['dialect_sum'].isin(list(range(2, 16)) + [18])]


balanced_dataset_2 = pd.DataFrame()

for value, group in filtered_dataset.groupby('dialect_sum'):
    if len(group) > desired_samples:
        group = resample(group, replace=False, n_samples=desired_samples, random_state=42)
    elif len(group) < desired_samples:
        group = resample(group, replace=True, n_samples=desired_samples, random_state=42)
    balanced_dataset_2 = pd.concat([balanced_dataset_2, group])

# Shuffle the dataset
balanced_dataset_2 = balanced_dataset_2.sample(frac=1, random_state=42).reset_index(drop=True)

print("Balanced dataset distribution:")
print(balanced_dataset_2['dialect_sum'].value_counts())


Balanced dataset distribution:
dialect_sum
3     1500
7     1500
14    1500
9     1500
12    1500
2     1500
11    1500
18    1500
10    1500
15    1500
4     1500
8     1500
6     1500
5     1500
13    1500
Name: count, dtype: int64


In [14]:
balanced_dataset['dialect_sum'].value_counts()
balanced_dataset = balanced_dataset[balanced_dataset['dialect_sum'] == 1]

In [15]:
new_dataset = pd.concat([balanced_dataset, balanced_dataset_2]).reset_index(drop=True)

In [16]:
new_dataset['dialect_sum'].value_counts()

dialect_sum
1     2700
3     1500
7     1500
14    1500
9     1500
12    1500
2     1500
11    1500
18    1500
10    1500
15    1500
4     1500
8     1500
6     1500
5     1500
13    1500
Name: count, dtype: int64

In [17]:
new_dataset.to_csv("Project/NADI2024/subtask1/multilabel/NADIcombined_oversampled.csv")

In [20]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
# dataset_path = f"/home/lara.hassan/Downloads/NADI2024_subtask1/subtask1/our_data/{file_name[4]}"
dataset_path = 'Project/NADI2024/subtask1/multilabel/NADIcombined_oversampled.csv'
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-mix",
    labels=labels,
    threshold=0.3,
    exp_num=26
)

trainer.save_dir = f'./exp_{trainer.exp_num}'

trainer.train(
    num_train_epochs=10,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.516000,0.361579,0.830442,0.835948,0.182143
2,0.323300,0.299061,0.855057,0.860643,0.249603
3,0.230900,0.272866,0.875414,0.882615,0.357937
4,0.174200,0.270055,0.883791,0.891444,0.438095
5,0.139200,0.282849,0.890102,0.897536,0.475397
6,0.115200,0.269263,0.898637,0.906114,0.516270
7,0.094600,0.287619,0.897030,0.904455,0.540079


In [30]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_24/marbert_finetuned_epochs_3_eval_f1_0.9034_greater_threshold_0.3/CAMeL-Lab-bert-base-arabic-camelbert-mix-experiment-24_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 70.54 %
MACRO AVERAGE RECALL SCORE: 44.46 %
MACRO AVERAGE F1-SCORE: 52.90 %
MACRO AVERAGE ACCURACY: 72.50 %



## Curriculum-based approach to balance the dataset exp 27-30

In [ ]:
dataset = pd.read_csv("Project/NADI2024/subtask1/multilabel/NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv")

In [10]:
dataset = dataset[dataset['dialect_sum'] == 1]

In [12]:
dataset.to_csv("Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_1.csv")

In [13]:
file_name = ["First_200.csv", "NADIcombined_cleaned_MULTI_LABEL_MODIFIED_FINAL.csv", "First_1000.csv", "balanced_multilabel_dataset.csv", "balanced_multilabel_dataset_500.csv"]
dataset_path = "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_1.csv"
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-mix",
    labels=labels,
    threshold=0.3,
    exp_num=27
)

trainer.save_dir = f'./exp_{trainer.exp_num}'

trainer.train(
    num_train_epochs=1,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 90%|█████████ | 503/557 [00:29<00:03, 17.69it/s]

{'loss': 0.2406, 'grad_norm': 0.5045507550239563, 'learning_rate': 5e-05, 'epoch': 0.9}


                                                 
100%|██████████| 557/557 [00:35<00:00, 11.21it/s]

{'eval_loss': 0.11656107008457184, 'eval_f1': 0.636464909023394, 'eval_roc_auc': 0.7819404228313038, 'eval_accuracy': 0.5664194200944033, 'eval_runtime': 1.2789, 'eval_samples_per_second': 1159.604, 'eval_steps_per_second': 48.48, 'epoch': 1.0}


100%|██████████| 557/557 [00:37<00:00, 14.79it/s]


{'train_runtime': 37.6679, 'train_samples_per_second': 354.307, 'train_steps_per_second': 14.787, 'train_loss': 0.2286419217847622, 'epoch': 1.0}
Subset Accuracy: 0.1750
Hamming Loss: 0.3500
Micro Precision: 0.7273
Micro Recall: 0.0899
Micro F1-Score: 0.1600
Precision per label: [0.         0.68421053 0.         0.         1.         1.
 1.         1.        ]
Recall per label: [0.         0.66666667 0.         0.         0.04761905 0.02173913
 0.0952381  0.01694915]
F1-Score per label: [0.         0.67532468 0.         0.         0.09090909 0.04255319
 0.17391304 0.03333333]
{0, 1}


/tmp/ipykernel_3325639/3611314930.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
dataset_path = "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_1.csv"
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name="CAMeL-Lab/bert-base-arabic-camelbert-mix",
    labels=labels,
    threshold=0.3,
    exp_num=27
)

trainer.save_dir = f'./exp_{trainer.exp_num}'

trainer.train(
    num_train_epochs=1,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

In [14]:
dataset

,id,tweet,Algeria,Bahrain,Egypt,Iraq,Jordan,Kuwait,Lebanon,Libya,...,Palestine,Qatar,Saudi_Arabia,Sudan,Syria,Tunisia,UAE,Yemen,Computed,dialect_sum
0,0,الفار العور يشوف فقط كيسي ومايشوف ماتويد,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,yes,1
6,6,هيتقال محروق ومش عارف تعمل زيهم,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,yes,1
7,7,والهوسات الزينه ابيت ماجد ياسين حمدالله سلامه ...,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,yes,1
8,8,انا الناس المتخلفه اللي بتعمل كده للاسف عايزه ...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,yes,1
12,12,مش عارف ليه حظي قليل اوي اتنشن انا وابويا برغم...,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,yes,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58735,58727,publié الحروف العربيه المقايس URL,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,yes,1
58737,58729,USER جشع وطمع اصحاب الشركات واستقواءهم علي الع...,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,yes,1
58738,58730,USER انت حلم كسرته كني اكسر ساقي حلم القدس رام...,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,yes,1
58744,58736,USER USER USER شكلها نست يوم ترامب قال للامريك...,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,yes,1


In [29]:
def balance_countries(n_samples,dataset):
    dialect_columns = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']


    rows_with_sum_1 = dataset[dataset['dialect_sum'] == 1]


    balanced_rows = pd.DataFrame()

    for dialect in dialect_columns:
     
        dialect_rows = rows_with_sum_1[rows_with_sum_1[dialect] == 1]
        
    
        resampled_rows = resample(dialect_rows, replace=True, 
                                n_samples=n_samples, 
                                random_state=42)
        balanced_rows = pd.concat([balanced_rows, resampled_rows])


    return balanced_rows

In [41]:
for i in range(2, 3):
    current_rows = dataset[dataset['dialect_sum'] == i]
    print(len(current_rows))
    for j in range(1, i):
        if j == 1:
            rows_with_sum_j = balance_countries(len(current_rows)//18, dataset)
        else:
            rows_with_sum_j = dataset[dataset["dialect_sum"] == j]
            if len(rows_with_sum_j) > len(current_rows):
                resampled_rows = resample(rows_with_sum_j, replace=True, 
                            n_samples=len(current_rows), 
                            random_state=42)
            else:
                resampled_rows = rows_with_sum_j
        current_rows = pd.concat([current_rows, resampled_rows])
        print(current_rows['dialect_sum'].value_counts())
        

3220
dialect_sum
2     3220
14     256
Name: count, dtype: int64


In [42]:
dataset

,id,tweet,Algeria,Bahrain,Egypt,Iraq,Jordan,Kuwait,Lebanon,Libya,...,Palestine,Qatar,Saudi_Arabia,Sudan,Syria,Tunisia,UAE,Yemen,Computed,dialect_sum
0,0,الفار العور يشوف فقط كيسي ومايشوف ماتويد,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,yes,1
1,1,ديني ربنا يستر,0,0,1,1,1,0,1,0,...,1,0,0,1,1,0,0,0,yes,7
2,2,اساسا نسبكم قذر ونجس بلاش تتفاخروا بنجاستكم وه...,0,0,1,1,1,0,1,0,...,1,0,1,0,1,0,0,1,yes,8
3,3,المشاعر تحتاج الي المشاعر تحتاج الي رفيق يخذل ...,0,0,1,0,1,0,1,0,...,1,0,0,0,0,0,0,0,yes,4
4,4,ني حاضرها لايف,0,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,yes,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58742,58734,جماعه الخير المنخفض الي جاي ايام منخفض قوي ماط...,0,0,0,0,1,0,1,0,...,1,0,0,0,1,0,0,0,yes,4
58743,58735,انا بايع الكل وشاري عيونك,0,0,1,0,1,0,0,0,...,1,0,1,1,0,0,0,0,yes,5
58744,58736,USER USER USER شكلها نست يوم ترامب قال للامريك...,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,yes,1
58745,58737,السطلات البنقو الحشيش معاكم URL ΉМĄDĄ,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,yes,1


In [56]:
from sklearn.utils import resample
import pandas as pd

def balance_countries(n_samples, dataset):
    dialect_columns = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
                       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
                       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']


    rows_with_sum_1 = dataset[dataset['dialect_sum'] == 1]

 
    balanced_rows = pd.DataFrame()


    for dialect in dialect_columns:
    
        dialect_rows = rows_with_sum_1[rows_with_sum_1[dialect] == 1]
        
        if len(dialect_rows) >= n_samples:
            resampled_rows = resample(dialect_rows, replace=False, 
                                      n_samples=n_samples, 
                                      random_state=42)
        else:
           
            resampled_rows = resample(dialect_rows, replace=True, 
                                      n_samples=n_samples, 
                                      random_state=42)
        
        balanced_rows = pd.concat([balanced_rows, resampled_rows], ignore_index=True)

    return balanced_rows



for i in list(range(2, 16)) + [18]:  
    
    current_rows = dataset[dataset['dialect_sum'] == i]
    print(f"Initial rows for dialect_sum = {i}: {len(current_rows)}")
    n = len(current_rows)
    for j in range(1, i):  
        if j == 1:
            
            resampled_rows = balance_countries(n// 18, dataset)
        else:
            
            rows_with_sum_j = dataset[dataset["dialect_sum"] == j]
            
            if len(rows_with_sum_j) > n:
                resampled_rows = resample(rows_with_sum_j, replace=True, 
                                          n_samples=n, 
                                          random_state=42)
            else:
                resampled_rows = rows_with_sum_j
        

        current_rows = pd.concat([current_rows, resampled_rows], ignore_index=True)
        print(f"Dialect_sum counts after combining with sum {j}:")
        print(current_rows['dialect_sum'].value_counts())
    current_rows.to_csv(f"Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_{i}.csv")
        
current_rows

Initial rows for dialect_sum = 2: 3220
Dialect_sum counts after combining with sum 1:
dialect_sum
2    3220
1    3204
Name: count, dtype: int64
Initial rows for dialect_sum = 3: 2329
Dialect_sum counts after combining with sum 1:
dialect_sum
3    2329
1    2322
Name: count, dtype: int64
Dialect_sum counts after combining with sum 2:
dialect_sum
3    2329
2    2329
1    2322
Name: count, dtype: int64
Initial rows for dialect_sum = 4: 2580
Dialect_sum counts after combining with sum 1:
dialect_sum
4    2580
1    2574
Name: count, dtype: int64
Dialect_sum counts after combining with sum 2:
dialect_sum
4    2580
2    2580
1    2574
Name: count, dtype: int64
Dialect_sum counts after combining with sum 3:
dialect_sum
4    2580
2    2580
1    2574
3    2329
Name: count, dtype: int64
Initial rows for dialect_sum = 5: 1416
Dialect_sum counts after combining with sum 1:
dialect_sum
5    1416
1    1404
Name: count, dtype: int64
Dialect_sum counts after combining with sum 2:
dialect_sum
5    1416


,id,tweet,Algeria,Bahrain,Egypt,Iraq,Jordan,Kuwait,Lebanon,Libya,...,Palestine,Qatar,Saudi_Arabia,Sudan,Syria,Tunisia,UAE,Yemen,Computed,dialect_sum
0,34,انا لله انا اليه راجعون الله يرحمه يصبر ذويه,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,yes,18
1,53,التقيكم قليل مباراه برشلونهريالمدريد كونوا علي...,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,yes,18
2,64,ولن يخلف الله وعده,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,yes,18
3,70,الله يرحمه ويرحم شهداءنا فخرنا وعزنا,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,yes,18
4,83,اللهم امين جميعا يارب,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,yes,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6706,8331,تجعلوا الله الها اخر اني نذير مبين الذاريات51,1,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,yes,17
6707,21928,يوم مليء بالمتخلفين كرويا يلا وقت نوم,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,yes,17
6708,23605,Ù Ø§ÙÙ Ø¯Ø®Ù Ø¨ÙÙÙ ÙÙÙÙ,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,yes,17
6709,51848,USER انا خلصت الزهايمر الاصلي وصلت لمرحله الزه...,1,1,1,1,1,1,1,0,...,1,1,1,1,1,1,1,1,yes,17


In [79]:
paths = ["Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_1.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_2.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_3.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_4.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_5.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_6.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_7.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_8.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_9.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_10.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_11.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_12.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_13.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_14.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_15.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_18.csv"]

stage = 0
dataset_path = paths[stage]
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name=f"CAMeL-Lab/bert-base-arabic-camelbert-mix",
    labels=labels,
    threshold=0.3,
    exp_num=28
)

trainer.save_dir = f'./exp_{trainer.exp_num}'

trainer.train(
    num_train_epochs=2,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at CAMeL-Lab/bert-base-arabic-camelbert-mix and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 35%|███▍      | 194/557 [00:53<00:20, 17.54it/s] 

{'loss': 0.2457, 'grad_norm': 0.4768322706222534, 'learning_rate': 5e-05, 'epoch': 0.9}














                                                  

 35%|███▍      | 194/557 [00:58<00:20, 17.54it/s]



{'eval_loss': 0.11737997084856033, 'eval_f1': 0.6376920194829524, 'eval_roc_auc': 0.7802744833604379, 'eval_accuracy': 0.5677680377612947, 'eval_runtime': 1.3101, 'eval_samples_per_second': 1131.972, 'eval_steps_per_second': 47.324, 'epoch': 1.0}



 35%|███▍      | 194/557 [01:28<00:20, 17.54it/s]  

...















                                                   

 35%|███▍      | 194/557 [01:40<00:20, 17.54it/s]



{'eval_loss': 0.10440827161073685, 'eval_f1': 0.6676166726042038, 'eval_roc_auc': 0.8082384673356867, 'eval_accuracy': 0.6210384356035064, 'eval_runtime': 1.9164, 'eval_samples_per_second': 773.838, 'eval_steps_per_second': 32.352, 'epoch': 2.0}



100%|██████████| 1114/1114 [01:17<00:00, 14.33it/s]


{'train_runtime': 77.725, 'train_samples_per_second': 343.416, 'train_steps_per_second': 14.333, 'train_loss': 0.16986481462817646, 'epoch': 2.0}
Subset Accuracy: 0.1750
Hamming Loss: 0.3458
Micro Precision: 0.7222
Micro Recall: 0.1096
Micro F1-Score: 0.1902
Precision per label: [1.         0.66666667 0.         1.         1.         1.
 1.         0.5       ]
Recall per label: [0.05714286 0.71794872 0.         0.046875   0.04761905 0.02173913
 0.0952381  0.01694915]
F1-Score per label: [0.10810811 0.69135802 0.         0.08955224 0.09090909 0.04255319
 0.17391304 0.03278689]
{0, 1, 2}


/tmp/ipykernel_3325639/3611314930.py:76: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [76]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_27/stage_16/-home-ali.mekky-Documents-NLP-Project-Cross-Country-Dialectal-Arabic-Identification-exp_27-stage_15-experiment-27_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 69.66 %
MACRO AVERAGE RECALL SCORE: 65.52 %
MACRO AVERAGE F1-SCORE: 65.49 %
MACRO AVERAGE ACCURACY: 76.04 %



In [83]:
for i in range(1, 16):
    paths = ["Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_1.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_2.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_3.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_4.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_5.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_6.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_7.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_8.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_9.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_10.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_11.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_12.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_13.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_14.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_15.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_18.csv"]

    stage = i
    dataset_path = paths[stage]
    dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
    labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
        'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
        'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

    trainer = BertTrainer(
        training_dataset_path=dataset_path,
        # model_name=f"CAMeL-Lab/bert-base-arabic-camelbert-mix",
        model_name=f"Project/Cross-Country-Dialectal-Arabic-Identification/exp_28/stage_{stage}",
        labels=labels,
        threshold=0.3,
        exp_num=28,
        stage = i + 1
    )

    trainer.save_dir = f'./exp_{trainer.exp_num}'

    trainer.train(
        num_train_epochs=2,
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        per_device_train_batch_size=24,
        per_device_eval_batch_size=24,
    )
    trainer.evaluate(dev_path=dev_path)

/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 50%|█████     | 241/482 [00:14<00:13, 17.63it/s]

{'eval_loss': 0.18958240747451782, 'eval_f1': 0.5371062466631074, 'eval_roc_auc': 0.7369372576095561, 'eval_accuracy': 0.26127527216174184, 'eval_runtime': 0.516, 'eval_samples_per_second': 1246.076, 'eval_steps_per_second': 52.324, 'epoch': 1.0}


100%|██████████| 482/482 [00:31<00:00, 17.90it/s]

{'eval_loss': 0.18514694273471832, 'eval_f1': 0.5550935550935551, 'eval_roc_auc': 0.7517128733128935, 'eval_accuracy': 0.2581648522550544, 'eval_runtime': 0.4763, 'eval_samples_per_second': 1350.01, 'eval_steps_per_second': 56.688, 'epoch': 2.0}


100%|██████████| 482/482 [00:33<00:00, 14.39it/s]


...


100%|██████████| 504/504 [00:32<00:00, 17.89it/s]

{'eval_loss': 0.38453784584999084, 'eval_f1': 0.8079353651707863, 'eval_roc_auc': 0.8113189757840414, 'eval_accuracy': 0.19345238095238096, 'eval_runtime': 0.519, 'eval_samples_per_second': 1294.735, 'eval_steps_per_second': 53.947, 'epoch': 2.0}


100%|██████████| 504/504 [00:34<00:00, 14.55it/s]


{'train_runtime': 34.6518, 'train_samples_per_second': 348.553, 'train_steps_per_second': 14.545, 'train_loss': 0.38235192308350213, 'epoch': 2.0}
Subset Accuracy: 0.1917
Hamming Loss: 0.2427
Micro Precision: 0.6762
Micro Recall: 0.6629
Micro F1-Score: 0.6695
Precision per label: [0.8        0.69565217 0.60869565 0.69642857 0.73076923 0.62962963
 0.6        0.72058824]
Recall per label: [0.34285714 0.82051282 0.84       0.609375   0.45238095 0.73913043
 0.42857143 0.83050847]
F1-Score per label: [0.48       0.75294118 0.70588235 0.65       0.55882353 0.68
 0.5        0.77165354]
{0, 1, 2, 3, 4, 5, 6, 7, 8}


/tmp/ipykernel_3325639/790495932.py:77: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [84]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_28/stage_16/-home-ali.mekky-Documents-NLP-Project-Cross-Country-Dialectal-Arabic-Identification-exp_28-stage_15-experiment-28_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 68.52 %
MACRO AVERAGE RECALL SCORE: 63.29 %
MACRO AVERAGE F1-SCORE: 63.74 %
MACRO AVERAGE ACCURACY: 75.73 %



## EXP 29

In [ ]:
"UBC-NLP/MARBERT"

In [85]:
paths = ["Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_1.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_2.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_3.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_4.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_5.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_6.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_7.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_8.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_9.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_10.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_11.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_12.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_13.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_14.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_15.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_18.csv"]

stage = 0
dataset_path = paths[stage]
dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
       'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
       'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

trainer = BertTrainer(
    training_dataset_path=dataset_path,
    model_name=f"UBC-NLP/MARBERT",
    labels=labels,
    threshold=0.3,
    exp_num=29,
    stage = 1
)

trainer.save_dir = f'./exp_{trainer.exp_num}'

trainer.train(
    num_train_epochs=1,
    metric_for_best_model="eval_f1",
    greater_is_better=True,
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
)
trainer.evaluate(dev_path=dev_path)
for i in range(1, 16):
    paths = ["Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_1.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_2.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_3.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_4.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_5.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_6.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_7.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_8.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_9.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_10.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_11.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_12.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_13.csv",
         "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_14.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_15.csv", "Project/Cross-Country-Dialectal-Arabic-Identification/CL_stages/stage_18.csv"]

    stage = i
    dataset_path = paths[stage]
    dev_path = "Project/NADI2024/subtask1/dev/NADI2024_subtask1_dev2.tsv"
    labels = ['Algeria', 'Bahrain', 'Egypt', 'Iraq', 'Jordan', 'Kuwait',
        'Lebanon', 'Libya', 'Morocco', 'Oman', 'Palestine', 'Qatar',
        'Saudi_Arabia', 'Sudan', 'Syria', 'Tunisia', 'UAE', 'Yemen']

    trainer = BertTrainer(
        training_dataset_path=dataset_path,
        model_name=f"Project/Cross-Country-Dialectal-Arabic-Identification/exp_29/stage_{stage}",
        labels=labels,
        threshold=0.3,
        exp_num=29,
        stage = i + 1
    )

    trainer.save_dir = f'./exp_{trainer.exp_num}'

    trainer.train(
        num_train_epochs=1,
        metric_for_best_model="eval_f1",
        greater_is_better=True,
        per_device_train_batch_size=24,
        per_device_eval_batch_size=24,
    )
    trainer.evaluate(dev_path=dev_path)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
 90%|█████████ | 502/557 [00:34<00:03, 14.51it/s]

{'loss': 0.2545, 'grad_norm': 1.5156006813049316, 'learning_rate': 5e-05, 'epoch': 0.9}


100%|██████████| 557/557 [00:42<00:00, 14.64it/s]

{'eval_loss': 0.1142585352063179, 'eval_f1': 0.6541635408852213, 'eval_roc_auc': 0.7878307088175797, 'eval_accuracy': 0.5812542144302091, 'eval_runtime': 1.2794, 'eval_samples_per_second': 1159.181, 'eval_steps_per_second': 48.462, 'epoch': 1.0}


100%|██████████| 557/557 [00:46<00:00, 12.09it/s]


...

/home/ali.mekky/.conda/envs/DAI/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
100%|██████████| 252/252 [00:21<00:00, 15.19it/s]

{'eval_loss': 0.3891410827636719, 'eval_f1': 0.8113332231951098, 'eval_roc_auc': 0.8180673547704116, 'eval_accuracy': 0.20089285714285715, 'eval_runtime': 0.5345, 'eval_samples_per_second': 1257.19, 'eval_steps_per_second': 52.383, 'epoch': 1.0}


100%|██████████| 252/252 [00:24<00:00, 10.19it/s]


{'train_runtime': 24.722, 'train_samples_per_second': 244.276, 'train_steps_per_second': 10.193, 'train_loss': 0.3943261948842851, 'epoch': 1.0}
Subset Accuracy: 0.1917
Hamming Loss: 0.2302
Micro Precision: 0.6912
Micro Recall: 0.6854
Micro F1-Score: 0.6883
Precision per label: [0.8        0.62       0.62686567 0.75925926 0.76666667 0.71698113
 0.5        0.72727273]
Recall per label: [0.34285714 0.79487179 0.84       0.640625   0.54761905 0.82608696
 0.42857143 0.81355932]
F1-Score per label: [0.48       0.69662921 0.71794872 0.69491525 0.63888889 0.76767677
 0.46153846 0.768     ]
{0, 1, 2, 3, 4, 5, 6, 7, 8}


/tmp/ipykernel_3325639/790495932.py:77: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_replaced = dev.replace({'y': 1, 'n': 0})


In [86]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_29/stage_16/-home-ali.mekky-Documents-NLP-Project-Cross-Country-Dialectal-Arabic-Identification-exp_29-stage_15-experiment-29_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 68.96 %
MACRO AVERAGE RECALL SCORE: 65.43 %
MACRO AVERAGE F1-SCORE: 65.32 %
MACRO AVERAGE ACCURACY: 76.98 %



In [9]:
stage = 16
trainer = BertTrainer(
    training_dataset_path=dataset_path,
    # model_name=f"CAMeL-Lab/bert-base-arabic-camelbert-mix",
    model_name=f"Project/Cross-Country-Dialectal-Arabic-Identification/exp_28/stage_{stage}",
    labels=labels,
    threshold=0.3,
    exp_num=29,
    # stage = i + 1
)

In [10]:
import os

scorer_script = "Project/NADI2024/subtask1/NADI2024-ST1-Scorer.py"
gold_file = "Project/NADI2024/subtask1/sample_submission/NADI2024_subtask1_dev2_gold.txt"
predictions_file = "Project/Cross-Country-Dialectal-Arabic-Identification/exp_28/stage_16/-home-ali.mekky-Documents-NLP-Project-Cross-Country-Dialectal-Arabic-Identification-exp_29-stage_15-experiment-28_predictions.txt"
!python3 "{scorer_script}" "{gold_file}" "{predictions_file}"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



OVERALL SCORES:
MACRO AVERAGE PRECISION SCORE: 68.96 %
MACRO AVERAGE RECALL SCORE: 65.43 %
MACRO AVERAGE F1-SCORE: 65.32 %
MACRO AVERAGE ACCURACY: 76.98 %

